In [1]:
import pandas as pd
import numpy as np
import numpy.linalg as la
from scipy import stats
from scipy.optimize import minimize
from tqdm import tqdm
from math import log, exp
from linearmodels import IV2SLS, IVGMM, IVGMMCUE, IVLIML


import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.panel import compare
from linearmodels.panel import FirstDifferenceOLS
from linearmodels.panel.data import PanelData
from linearmodels.panel import PanelOLS
from linearmodels.panel import BetweenOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import PooledOLS
from statsmodels.stats.diagnostic import het_white, het_breuschpagan
from linearmodels.panel import compare
from optimask import OptiMask
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import sys
sys.path.append(r'C:\Users\jumer\Documents\Documents Co\VSCode\EPS_sectoral\eps_package\src')

from eps.EPS import EPS
from eps.settings import *

%load_ext autoreload
%autoreload 2



In [2]:
eps_class = EPS(
    econo_computed=False
)
eps_class()

invalid value encountered in log
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.

# Plots

## EPS plots 

### Settings 

In [ ]:
countries_to_drop_plots = ['USA', 'BRA', 'MEX', 'SAU','ZAF']
EPS_total = (eps_class.EPS_total
            #.dropna(axis = 0)
            .query(f"~iso3.isin({countries_to_drop_plots})")
)
colors_sector = ['#cc0000', '#0b5394', '#f1c232', '#6aa84f']
colors_categorie = ['#990000', '#38761d']
colors_year = ['#6DC6BF', '#A4CB90']

dc_col_to_name = {
    'EPS_BOD': 'EPS BOD Value',
    'EPS_OCDE': 'EPS OCDE Value',
    'EPS_GHG': 'EPS GHG Value',
    'EPS_GDP' : 'EPS GDP Value',
    'EPS_BOD_100var' : 'EPS BOD Value'
}

target = 'EPS_BOD_100var'
year1 = 2020 # Year to order by
year2 = 2000

### EPS_total

In [ ]:
#graph EPS/Year
df_plot = (EPS_total
    .query("sector == 'total'")
    .groupby('Year')[['EPS_OCDE', 'EPS_GDP','EPS_GHG', 'EPS_BOD_100var']].mean().reset_index()
    .melt(id_vars='Year', var_name='EPS Type', value_name='EPS_Value')
)

fig = px.line(df_plot, x='Year', y='EPS_Value', color='EPS Type')
fig.update_yaxes(title='EPS Value')
fig.show()

In [ ]:
# Maps per country

df_plot = (EPS_total
    .query("sector == 'total'")
)

fig = px.choropleth(df_plot, locations='iso3', color=target , color_continuous_scale=px.colors.sequential.Plasma, animation_frame='Year')
# fig.write_html(r"C:\Users\B25880\OneDrive - EDF\Documents drive\VSCode\map.html")
fig.show()

In [ ]:
#EPS/country in 2000 - 2020
df_plot = (EPS_total
    .query("sector == 'total'")
    .query(f"Year.isin([{year1}, {year2}])")
    .loc[:, ['iso3', 'Year', target]]
    .assign(Year = lambda df: df.Year.astype('str'))
)

fig = px.bar(df_plot, x='iso3', y=target, color='Year', barmode='group', color_discrete_sequence=['#6DC6BF', '#A4CB90'])

# Trier le DataFrame par EPS_bod pour l'année 2020
df_plot_2020 = df_plot[df_plot['Year'] == str(year1)].sort_values(by=target)
iso3_order = df_plot_2020['iso3'].tolist()

# Mettre à jour l'ordre des catégories sur l'axe des x pour l'année 2020
fig.update_xaxes(categoryorder='array', categoryarray=iso3_order)
fig.update_yaxes(title = dc_col_to_name[target])

fig.show()

### MBI/NMBI

In [ ]:
# By Year
#target= 'EPS_GHG'
df_plot = (EPS_total
    .query("categorie != 'total'")
    .groupby(['Year', 'iso3', 'categorie'], as_index=False)
    .agg(EPS_value = (target, 'sum'))
    .groupby(['Year', 'categorie'], as_index=False)
    .agg(EPS_value = ('EPS_value', 'mean'))
)

df_plot

fig = px.bar(df_plot, x='Year', y='EPS_value', color='categorie', color_discrete_sequence=colors_categorie)
fig.update_yaxes(title = dc_col_to_name[target])
fig.show()

In [ ]:
# By country

df_plot = (EPS_total
    .query("categorie != 'total'")
    .query(f"Year == {year1}")
    .groupby(['Year', 'iso3', 'categorie'], as_index=False)
    .agg(EPS_value = (target, 'sum'))
)

fig = px.bar(df_plot, x='iso3', y='EPS_value', color='categorie', color_discrete_sequence=colors_categorie)

iso3_order = list(df_plot
    .groupby('iso3', as_index=False)
    .agg(EPS_value = ('EPS_value', 'sum'))
    .sort_values('EPS_value')
    .iso3
)

fig.update_xaxes(categoryorder='array', categoryarray=iso3_order)
fig.update_yaxes(title = dc_col_to_name[target])
fig.show()

### By Sector

In [ ]:
# Focus on a specific country

iso3 = 'FRA'

df_plot = (EPS_total
    .query("sector != 'total' and categorie == 'total'")
    .query(f"iso3 == '{iso3}'")
    # .groupby(['Year', 'sector'], as_index=False)
    # .agg(EPS_value = (target, 'mean'))
)

fig = px.line(df_plot, x='Year', y=target, color='sector', color_discrete_sequence=colors_sector, title=f"Evolution of {dc_col_to_name[target]} for {iso3} over the years")
fig.update_yaxes(title = dc_col_to_name[target])
fig.show()

In [ ]:
target = 'EPS_BOD'
df_plot = (EPS_total
    .query(f"categorie == 'total' and sector != 'total' and Year == {year1}")
)
df_plot

iso3_order = list(df_plot
    .groupby('iso3', as_index=False)
    .agg(EPS_value = (target, 'mean'))
    .sort_values('EPS_value')
    .iso3
)

fig = px.bar(df_plot, x='iso3', y=target, color='sector', color_discrete_sequence=colors_sector)
fig.update_yaxes(title = dc_col_to_name[target])
fig.update_xaxes(categoryorder='array', categoryarray=iso3_order)
fig.show()

## GHG and GDP plots 

### Settings

In [ ]:
countries_to_drop_plots = ['USA', 'BRA', 'MEX', 'SAU','ZAF']
df = (eps_class.econo
      .dropna(axis = 0)
      .query(f"~iso3.isin({countries_to_drop_plots})")
) 

df['GHG_per_GDP'] = df['GHG'] / df['GDP']*100
df['ln_GHG_per_GDP'] = np.log(df['GHG_per_GDP'])
colors_variable = ['#582900', '#A9A9A9','#000000']
colors_sector = ['#cc0000', '#0b5394', '#f1c232', '#6aa84f','#000000']
year1 = 2020 # Year to order by
year2 = 2000

In [ ]:
#graph GHG/Year
df_plot = (df
    .groupby('Year')[['ln_GDP', 'ln_POP',  'ln_GHG']].mean().reset_index()
    .melt(id_vars='Year', var_name='variable', value_name='variable_Value')
)

fig = px.line(df_plot, x='Year', y='variable_Value', color='variable', color_discrete_sequence=colors_variable)
fig.update_yaxes(title='Control and GHG trends')
fig.show()

In [ ]:
#graph GHG/Year
df_plot = (df
    .groupby('Year')[['ln_building_GHG', 'ln_elec_GHG', 'ln_indus_GHG', 'ln_transport_GHG','ln_GHG']].mean().reset_index()
    .melt(id_vars='Year', var_name='variable', value_name='variable_Value')
)

fig = px.line(df_plot, x='Year', y='variable_Value', color='variable', color_discrete_sequence=colors_sector)
fig.update_yaxes(title='ln_GHG trends')
fig.show()

In [ ]:
# Grouper par année et calculer la moyenne du ratio GHG/GDP
df_plot = (df
    .groupby('Year')[['GHG_per_GDP']].mean().reset_index()
    .melt(id_vars='Year', var_name='variable', value_name='variable_Value')
)

# Créer le graphique de ligne
fig = px.line(df_plot, x='Year', y='variable_Value', color='variable')
fig.update_yaxes(title='GHG per GDP trends')
fig.update_layout(title='Trends of GHG per GDP over the Years')
fig.show()

In [ ]:
# Grouper par année et calculer la moyenne du ln(GHG/GDP)
df_plot = (df
    .groupby('Year')[['ln_GHG_per_GDP']].mean().reset_index()
    .melt(id_vars='Year', var_name='variable', value_name='variable_Value')
)

# Créer le graphique de ligne
fig = px.line(df_plot, x='Year', y='variable_Value', color='variable')
fig.update_yaxes(title='Log of GHG per GDP trends')
fig.update_layout(title='Trends of Log(GHG per GDP) over the Years')
fig.show()

# Econometrics

## Variables

### BOD

In [3]:
econo_var = [
    'EPS_BOD_100var',
    'EPS_BOD_100var_building_mbi',
    'EPS_BOD_100var_building_nmbi', 
    'EPS_BOD_100var_building',
    'EPS_BOD_100var_elec_mbi',
    'EPS_BOD_100var_elec_nmbi',
    'EPS_BOD_100var_elec',
    'EPS_BOD_100var_indus_mbi',
    'EPS_BOD_100var_indus_nmbi',
    'EPS_BOD_100var_indus',
    'EPS_BOD_100var_transport_mbi',
    'EPS_BOD_100var_transport_nmbi', 
    'EPS_BOD_100var_transport',
    'EPS_BOD_100var_mbi_mbi',
    'EPS_BOD_100var_nmbi_nmbi',
    'EPS_BOD_100var_building_lag',
    'EPS_BOD_100var_elec_lag',
    'EPS_BOD_100var_indus_lag',
    'EPS_BOD_100var_transport_lag',
    'ln_GDP',
    'GDP_growth',
    'GDP_growth_2',
    'ln_GDP_2',
    'ln_GDP_per_cap_2',
    'ln_POP',
    'GHG',
    'GDP',
    'ln_building_GHG',
    'ln_elec_GHG',
    'ln_indus_GHG',
    'ln_GHG',
    'ln_transport_GHG',
#     'FDI',
    'urban_growth',
    'ln_GDP_per_cap',
    'POP_growth',
    'ln_building_GHG_per_cap',
    'ln_elec_GHG_per_cap',
    'ln_indus_GHG_per_cap',
    'ln_GHG_per_cap',
    'ln_transport_GHG_per_cap',
    'agri_0',
    'ETS_E',
    'ETS_T',
    'ETS_B',
    'ETS_I',
    'ETS'
]




eps_class.clean_econo_database(
    model_name = 'BOD',
    selection_type = 'reference', # ['max_year', 'max_iso3', 'max_obs', 'reference']
    econo_var = econo_var,
    save_dataframe = True,
    already_computed = False
)
BOD_df=eps_class.preprocessed_econo
#.query("groupe2 == 'Developing'")
#.query("groupe1 == 'EU'")
BOD_df['urban_growth'] = BOD_df['urban_growth'].astype(float)

modBOD_tot = BOD_df.loc[:, ['iso3', 'Year'] + econo_var]

# Création des variables ln_EPS_sector
modBOD_tot['ln_EPS_BOD_100var'] = np.log(modBOD_tot['EPS_BOD_100var'] + 1)
modBOD_tot['ln_EPS_BOD_100var_building'] = np.log(modBOD_tot['EPS_BOD_100var_building'] + 1)
modBOD_tot['ln_EPS_BOD_100var_elec'] = np.log(modBOD_tot['EPS_BOD_100var_elec'] + 1)
modBOD_tot['ln_EPS_BOD_100var_indus'] = np.log(modBOD_tot['EPS_BOD_100var_indus'] + 1)
modBOD_tot['ln_EPS_BOD_100var_transport'] = np.log(modBOD_tot['EPS_BOD_100var_transport'] + 1)

modBOD_tot['tr_EPS_BOD_100var'] = modBOD_tot['EPS_BOD_100var'] + 1
modBOD_tot['tr_EPS_BOD_100var_building'] = modBOD_tot['EPS_BOD_100var'] + 1
modBOD_tot['tr_EPS_BOD_100var_elec'] = modBOD_tot['EPS_BOD_100var'] + 1
modBOD_tot['tr_EPS_BOD_100var_indus'] = modBOD_tot['EPS_BOD_100var'] + 1
modBOD_tot['tr_EPS_BOD_100var_transport'] = modBOD_tot['EPS_BOD_100var'] + 1

In [ ]:
# Summary Statistics

# Création d'un DataFrame avec les variables spécifiées
data = modBOD_tot[econo_var]

# Calculer les statistiques descriptives pour chaque variable
descriptive_stats = data.describe().T

# Formater les statistiques descriptives comme un tableau
table = descriptive_stats.to_string()

# Afficher le tableau formaté
print(table)

### OCDE

In [4]:
econo_var = [
    'EPS_OCDE',
    'EPS_OCDE_building_mbi',
    'EPS_OCDE_building_nmbi', 
    'EPS_OCDE_building',
    'EPS_OCDE_building_lag',
    'EPS_OCDE_elec_mbi',
    'EPS_OCDE_elec_nmbi',
    'EPS_OCDE_elec',
    'EPS_OCDE_elec_lag',
    'EPS_OCDE_indus_mbi',
    'EPS_OCDE_indus_nmbi',
    'EPS_OCDE_indus',
    'EPS_OCDE_indus_lag',
    'EPS_OCDE_transport_mbi',
    'EPS_OCDE_transport_nmbi', 
    'EPS_OCDE_transport',
    'EPS_OCDE_transport_lag',
    'EPS_OCDE_mbi_mbi',
    'EPS_OCDE_nmbi_nmbi',
    'ln_GDP',
    'ln_GDP_2',
    'GDP_growth',
    'GDP_growth_2',
    'ln_POP',
    'ln_building_GHG',
    'ln_elec_GHG',
    'ln_indus_GHG',
    'GHG',
    'GDP',
    'ln_GHG',
    'ln_transport_GHG',
    'urban_growth',
    'ln_GDP_per_cap',
    'ln_GDP_per_cap_2',
    'POP_growth',
    'ln_GHG_per_cap',
    'agri_0',
    'ETS_E',
    'ETS_T',
    'ETS_B',
    'ETS_I',
    'ETS'
]

eps_class.clean_econo_database(
    model_name = 'OCDE',
    selection_type = 'reference', # ['max_year', 'max_iso3', 'max_obs', 'reference']
    econo_var = econo_var,
    save_dataframe = True,
    already_computed = False
)
OCDE_df=eps_class.preprocessed_econo
#.query("groupe2 == 'Developing'")
#.query("groupe1 == 'EU'")
OCDE_df['urban_growth'] = OCDE_df['urban_growth'].astype(float)

modOCDE_tot = OCDE_df.loc[:, ['iso3', 'Year'] + econo_var]

#modOCDE_ETS_E = modOCDE[modOCDE['ETS'] == 1]
# Création des variables ln_EPS_sector
modOCDE_tot['ln_EPS_OCDE'] = np.log(modOCDE_tot['EPS_OCDE'] + 1)
modOCDE_tot['ln_EPS_OCDE_building'] = np.log(modOCDE_tot['EPS_OCDE_building'] + 1)
modOCDE_tot['ln_EPS_OCDE_elec'] = np.log(modOCDE_tot['EPS_OCDE_elec'] + 1)
modOCDE_tot['ln_EPS_OCDE_indus'] = np.log(modOCDE_tot['EPS_OCDE_indus'] + 1)
modOCDE_tot['ln_EPS_OCDE_transport'] = np.log(modOCDE_tot['EPS_OCDE_transport'] + 1)

modOCDE_tot['tr_EPS_OCDE'] = modOCDE_tot['EPS_OCDE'] + 1
modOCDE_tot['tr_EPS_OCDE_building'] = modOCDE_tot['EPS_OCDE'] + 1
modOCDE_tot['tr_EPS_OCDE_elec'] = modOCDE_tot['EPS_OCDE'] + 1
modOCDE_tot['tr_EPS_OCDE_indus'] = modOCDE_tot['EPS_OCDE'] + 1
modOCDE_tot['tr_EPS_OCDE_transport'] = modOCDE_tot['EPS_OCDE'] + 1


In [ ]:
# Summary Statistics

# Création d'un DataFrame avec les variables spécifiées
data = modOCDE_tot[econo_var]

# Calculer les statistiques descriptives pour chaque variable
descriptive_stats = data.describe().T

# Formater les statistiques descriptives comme un tableau
table = descriptive_stats.to_string()

# Afficher le tableau formaté
print(table)

### GHG

In [5]:
econo_var = [
    'EPS_GHG',
    'EPS_GHG_building_mbi',
    'EPS_GHG_building_nmbi', 
    'EPS_GHG_building',
    'EPS_GHG_elec_mbi',
    'EPS_GHG_elec_nmbi',
    'EPS_GHG_elec',
    'EPS_GHG_indus_mbi',
    'EPS_GHG_indus_nmbi',
    'EPS_GHG_indus',
    'EPS_GHG_transport_mbi',
    'EPS_GHG_transport_nmbi', 
    'EPS_GHG_transport',
    'EPS_GHG_mbi_mbi',
    'EPS_GHG_nmbi_nmbi',
    'EPS_GHG_building_lag',
    'EPS_GHG_elec_lag',
    'EPS_GHG_indus_lag',
    'EPS_GHG_transport_lag',
    'ln_GDP',
    'ln_GDP_2',
    'GDP_growth',
    'GDP_growth_2',
    'ln_GDP_per_cap_2',
    'GHG',
    'GDP',
    'ln_POP',
    'ln_building_GHG',
    'ln_elec_GHG',
    'ln_indus_GHG',
    'ln_GHG',
    'ln_transport_GHG',
    'urban_growth',
    'ln_GDP_per_cap',
    'POP_growth',
    'ln_GHG_per_cap',
    'agri_0',
    'ETS_E',
    'ETS_T',
    'ETS_B',
    'ETS_I',
    'ETS'
]

# econo_var = [
#     'EPS_OCDE',
#     'EPS_BOD_100var',
#     'EPS_GHG',
#     'EPS_GDP',
#     'ln_GDP',
#     'ln_POP',
#     'ln_GHG',
#     'urban_growth'
# ]






eps_class.clean_econo_database(
    model_name = 'GHG',
    selection_type = 'reference', # ['max_year', 'max_iso3', 'max_obs', 'reference']
    econo_var = econo_var,
    save_dataframe = True,
    already_computed = False
)
GHG_df=eps_class.preprocessed_econo
#.query("groupe2 == 'Developing'")
#.query("groupe1 == 'EU'")
GHG_df['urban_growth'] = GHG_df['urban_growth'].astype(float)

modGHG_tot = GHG_df.loc[:, ['iso3', 'Year'] + econo_var]

modGHG_tot['ln_EPS_GHG'] = np.log(modGHG_tot['EPS_GHG'] + 1)
modGHG_tot['ln_EPS_GHG_building'] = np.log(modGHG_tot['EPS_GHG_building'] + 1)
modGHG_tot['ln_EPS_GHG_elec'] = np.log(modGHG_tot['EPS_GHG_elec'] + 1)
modGHG_tot['ln_EPS_GHG_indus'] = np.log(modGHG_tot['EPS_GHG_indus'] + 1)
modGHG_tot['ln_EPS_GHG_transport'] = np.log(modGHG_tot['EPS_GHG_transport'] + 1)

modGHG_tot['tr_EPS_GHG'] = modGHG_tot['EPS_GHG'] + 1
modGHG_tot['tr_EPS_GHG_building'] = modGHG_tot['EPS_GHG'] + 1
modGHG_tot['tr_EPS_GHG_elec'] = modGHG_tot['EPS_GHG'] + 1
modGHG_tot['tr_EPS_GHG_indus'] = modGHG_tot['EPS_GHG'] + 1
modGHG_tot['tr_EPS_GHG_transport'] = modGHG_tot['EPS_GHG'] + 1

In [ ]:
# Summary Statistics

# Création d'un DataFrame avec les variables spécifiées
data = modGHG_tot[econo_var]

# Calculer les statistiques descriptives pour chaque variable
descriptive_stats = data.describe().T

# Formater les statistiques descriptives comme un tableau
table = descriptive_stats.to_string()

# Afficher le tableau formaté
print(table)

### GDP

In [6]:

econo_var = [
    'EPS_GDP',
    'EPS_GDP_building_mbi',
    'EPS_GDP_building_nmbi', 
    'EPS_GDP_building',
    'EPS_GDP_elec_mbi',
    'EPS_GDP_elec_nmbi',
    'EPS_GDP_elec',
    'EPS_GDP_indus_mbi',
    'EPS_GDP_indus_nmbi',
    'EPS_GDP_indus',
    'EPS_GDP_transport_mbi',
    'EPS_GDP_transport_nmbi', 
    'EPS_GDP_transport',
    'EPS_GDP_mbi_mbi',
    'EPS_GDP_nmbi_nmbi',
    'EPS_GDP_building_lag',
    'EPS_GDP_elec_lag',
    'EPS_GDP_indus_lag',
    'EPS_GDP_transport_lag',
    'ln_GDP',
    'ln_GDP_2',
    'GDP_growth',
    'GDP_growth_2',
    'ln_GDP_per_cap_2',
    'GHG',
    'GDP',
    'ln_POP',
    'ln_building_GHG',
    'ln_elec_GHG',
    'ln_indus_GHG',
    'ln_GHG',
    'ln_transport_GHG',
    'urban_growth',
    'ln_GDP_per_cap',
    'POP_growth',
    'ln_building_GHG_per_cap',
    'ln_elec_GHG_per_cap',
    'ln_indus_GHG_per_cap',
    'ln_GHG_per_cap',
    'ln_transport_GHG_per_cap',
    'agri_0',
    'ETS_E',
    'ETS_T',
    'ETS_B',
    'ETS_I',
    'ETS'
]


eps_class.clean_econo_database(
    model_name = 'GDP',
    selection_type = 'reference', # ['max_year', 'max_iso3', 'max_obs', 'reference']
    econo_var = econo_var,
    save_dataframe = True,
    already_computed = False
)
GDP_df=eps_class.preprocessed_econo
#.query("groupe2 == 'Developing'")
#.query("groupe1 == 'EU'")
GDP_df['urban_growth'] = GDP_df['urban_growth'].astype(float)

modGDP_tot = GDP_df.loc[:, ['iso3', 'Year'] + econo_var]

modGDP_tot['ln_EPS_GDP'] = np.log(modGDP_tot['EPS_GDP'] + 1)
modGDP_tot['ln_EPS_GDP_building'] = np.log(modGDP_tot['EPS_GDP_building'] + 1)
modGDP_tot['ln_EPS_GDP_elec'] = np.log(modGDP_tot['EPS_GDP_elec'] + 1)
modGDP_tot['ln_EPS_GDP_indus'] = np.log(modGDP_tot['EPS_GDP_indus'] + 1)
modGDP_tot['ln_EPS_GDP_transport'] = np.log(modGDP_tot['EPS_GDP_transport'] + 1)

modGDP_tot['tr_EPS_GDP'] = modGDP_tot['EPS_GDP'] + 1
modGDP_tot['tr_EPS_GDP_building'] = modGDP_tot['EPS_GDP'] + 1
modGDP_tot['tr_EPS_GDP_elec'] = modGDP_tot['EPS_GDP'] + 1
modGDP_tot['tr_EPS_GDP_indus'] = modGDP_tot['EPS_GDP'] + 1
modGDP_tot['tr_EPS_GDP_transport'] = modGDP_tot['EPS_GDP'] + 1

In [ ]:
# Summary Statistics

# Création d'un DataFrame avec les variables spécifiées
data = modGDP_tot[econo_var]

# Calculer les statistiques descriptives pour chaque variable
descriptive_stats = data.describe().T

# Formater les statistiques descriptives comme un tableau
table = descriptive_stats.to_string()

# Afficher le tableau formaté
print(table)

### Clean data for homogeneous df (number of obs) between EPS

In [7]:
# Colonnes d'intérêt (celles utilisées dans la régression)
columns_of_interest_gdp = ['ln_GHG', 'ln_GHG', 'ln_GHG', 'ln_GHG','ln_GDP', 'ln_POP', 'urban_growth',
                            'EPS_GDP']
columns_of_interest_ghg = ['ln_GHG', 'ln_GHG', 'ln_GHG', 'ln_GHG','ln_GDP', 'ln_POP', 'urban_growth',
                            'EPS_GHG']

missing_keys = pd.concat([
    modGDP_tot[modGDP_tot[columns_of_interest_gdp].isnull().any(axis=1)][['iso3', 'Year']],
    modGHG_tot[modGHG_tot[columns_of_interest_ghg].isnull().any(axis=1)][['iso3', 'Year']]
]).drop_duplicates()

In [8]:
modBOD = modBOD_tot[~modBOD_tot[['iso3', 'Year']].apply(tuple,axis=1).isin(missing_keys.apply(tuple, axis=1))]
modOCDE = modOCDE_tot[~modOCDE_tot[['iso3', 'Year']].apply(tuple,axis=1).isin(missing_keys.apply(tuple, axis=1))]
modGHG = modGHG_tot[~modGHG_tot[['iso3', 'Year']].apply(tuple,axis=1).isin(missing_keys.apply(tuple, axis=1))]
modGDP = modGDP_tot[~modGDP_tot[['iso3', 'Year']].apply(tuple,axis=1).isin(missing_keys.apply(tuple, axis=1))]

In [ ]:
# Summary Statistics
sum_stat = ['ln_GDP', 'ln_POP', 'urban_growth', 'agri_0', 'ln_GHG', 
               'EPS_GDP_building', 'EPS_GDP_elec', 'EPS_GDP_indus', 
               'EPS_GDP_transport', 'EPS_GDP']
# Création d'un DataFrame avec les variables spécifiées
data = modGDP[sum_stat]

# Calculer les statistiques descriptives pour chaque variable
descriptive_stats = data.describe().T

# Formater les statistiques descriptives comme un tableau
table = descriptive_stats.to_string()

# Afficher le tableau formaté
print(table)

In [14]:
print(modOCDE.loc[modOCDE['ETS_T'] != 0, 'iso3'])

iso3  Year
NZL   2010    NZL
      2011    NZL
      2012    NZL
CAN   2013    CAN
NZL   2013    NZL
CAN   2014    CAN
NZL   2014    NZL
CAN   2015    CAN
NZL   2015    NZL
CAN   2016    CAN
NZL   2016    NZL
CAN   2017    CAN
NZL   2017    NZL
CAN   2018    CAN
NZL   2018    NZL
CAN   2019    CAN
NZL   2019    NZL
CAN   2020    CAN
NZL   2020    NZL
Name: iso3, dtype: category
Categories (51, object): ['ARG', 'AUS', 'AUT', 'BEL', ..., 'SVN', 'SWE', 'TUR', 'ZAF']


## IV Bartik

### Instrument

In [10]:
# Define columns to keep
col_to_keep = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'agri_0', 'ln_GHG', 
               'EPS_OCDE_building', 'EPS_OCDE_elec', 'EPS_OCDE_indus', 
               'EPS_OCDE_transport', 'Year_col', 'iso3_col', 'EPS_OCDE']

# Create the entire process in a single chain
bartik_df = (
    modOCDE
    # Step 1: Assign iso3_col and Year_col from index
    .assign(
        iso3_col=lambda df: df.index.get_level_values(0),
        Year_col=lambda df: df.index.get_level_values(1)
    )
    
    # Step 2: Filter columns to keep
    .loc[:, ['iso3', 'Year'] + col_to_keep]
    
    # Step 3: Calculate agri_0_share (constant share for each country)
    .assign(
        agri_0_share=lambda df: df.groupby('iso3_col')['agri_0'].transform('first')
    )
    
    .query("Year_col > 1990")

    # Step 4: Create all combinations of countries and years
    .pipe(lambda df: pd.MultiIndex.from_product([df['iso3_col'].unique(), 
                                                 pd.Series(range(df['Year_col'].min(), df['Year_col'].max() + 1))], 
                                                names=['iso3_col', 'Year_col'])
                      .to_frame(index=False)
                      .merge(df, on=['iso3_col', 'Year_col'], how='left')
                      .sort_values(['iso3_col', 'Year_col']))
    
    # Step 5: Forward fill EPS_GDP values by country and fill remaining NaNs with 0 for each EPS variable
    .assign(
        EPS_OCDE=lambda df: df.groupby('iso3_col')['EPS_OCDE'].ffill().fillna(0),
        EPS_OCDE_building=lambda df: df.groupby('iso3_col')['EPS_OCDE_building'].ffill().fillna(0),
        EPS_OCDE_elec=lambda df: df.groupby('iso3_col')['EPS_OCDE_elec'].ffill().fillna(0),
        EPS_OCDE_indus=lambda df: df.groupby('iso3_col')['EPS_OCDE_indus'].ffill().fillna(0),
        EPS_OCDE_transport=lambda df: df.groupby('iso3_col')['EPS_OCDE_transport'].ffill().fillna(0)
    )
    
    # Step 6: Pre-calculate total EPS and count per year for each EPS variable
    .assign(
        total_eps_per_year=lambda df: df.groupby('Year_col')['EPS_OCDE'].transform('sum'),
        country_count_per_year=lambda df: df.groupby('Year_col')['EPS_OCDE'].transform('count'),
        total_eps_building_per_year=lambda df: df.groupby('Year_col')['EPS_OCDE_building'].transform('sum'),
        total_eps_elec_per_year=lambda df: df.groupby('Year_col')['EPS_OCDE_elec'].transform('sum'),
        total_eps_indus_per_year=lambda df: df.groupby('Year_col')['EPS_OCDE_indus'].transform('sum'),
        total_eps_transport_per_year=lambda df: df.groupby('Year_col')['EPS_OCDE_transport'].transform('sum')
    )
    
    # Step 7: Calculate EPS shift (average EPS for other countries) for each EPS variable
    .assign(
        EPS_shift=lambda df: (df['total_eps_per_year'] - df['EPS_OCDE']) / (df['country_count_per_year'] - 1),
        EPS_shift_building=lambda df: (df['total_eps_building_per_year'] - df['EPS_OCDE_building']) / (df['country_count_per_year'] - 1),
        EPS_shift_elec=lambda df: (df['total_eps_elec_per_year'] - df['EPS_OCDE_elec']) / (df['country_count_per_year'] - 1),
        EPS_shift_indus=lambda df: (df['total_eps_indus_per_year'] - df['EPS_OCDE_indus']) / (df['country_count_per_year'] - 1),
        EPS_shift_transport=lambda df: (df['total_eps_transport_per_year'] - df['EPS_OCDE_transport']) / (df['country_count_per_year'] - 1)
    )
    
    # Step 8: Calculate Bartik instrument for each EPS variable
    .assign(
        Bartik_instrument=lambda df: df['agri_0_share'] * df['EPS_shift'],
        Bartik_instrument_building=lambda df: df['agri_0_share'] * df['EPS_shift_building'],
        Bartik_instrument_elec=lambda df: df['agri_0_share'] * df['EPS_shift_elec'],
        Bartik_instrument_indus=lambda df: df['agri_0_share'] * df['EPS_shift_indus'],
        Bartik_instrument_transport=lambda df: df['agri_0_share'] * df['EPS_shift_transport']
    )
    
    # Step 10: Set final index
    .set_index(['iso3_col', 'Year_col'])
)


In [11]:
df_bartik = bartik_df.join([
    modBOD[['EPS_BOD_100var']], 
    modGHG[['EPS_GHG']], 
    modGDP[['EPS_GDP']]
], how='left')

#### Cov. Est. Test

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey
endog = df_bartik['EPS_OCDE']
exog_var = df_bartik[['ln_GDP', 'ln_POP', 'urban_growth', 'Bartik_instrument']]
exog = sm.add_constant(exog_var)
model_ba = PanelOLS(endog, exog, entity_effects=True, time_effects=True) 
ba_res = model_ba.fit(cov_type='clustered')

df_bartik['pred_EPS'] = ba_res.predict().reindex(df_bartik.index).values

# Second-stage regression
endog = df_bartik['ln_GHG']
exog_var = df_bartik[['ln_GDP', 'ln_POP', 'urban_growth', 'pred_EPS']]
exog_2 = sm.add_constant(exog_var)
model_ba = PanelOLS(endog, exog, entity_effects=True, time_effects=True)
ba_res_2 = model_ba.fit(cov_type='unadjusted')



In [ ]:
exog_clean = exog_2.dropna().replace([np.inf, -np.inf], np.nan).dropna()

In [ ]:
### 1. Heteroskedasticity Test (Breusch-Pagan)
# Breusch-Pagan requires the residuals and the independent variables (exog)
_, bp_pvalue, _, _ = het_breuschpagan(ba_res.resids, exog_clean)
print(f"Breusch-Pagan test for heteroskedasticity - p-value: {bp_pvalue}")
if bp_pvalue < 0.05:
    print("There is evidence of heteroskedasticity (use robust or clustered standard errors).")
else:
    print("No evidence of heteroskedasticity.")

In [ ]:
from statsmodels.regression.linear_model import OLS
residuals = ba_res.resids

# Step 2: Perform OLS regression of residuals on lags of residuals (like time-series)
lags = 2  # You can adjust the number of lags
bg_test = acorr_breusch_godfrey(OLS(residuals, exog_clean).fit(), nlags=lags)
print(f"Breusch-Godfrey test for autocorrelation - p-value: {bg_test[1]}")
if bg_test[1] < 0.05:
    print("There is evidence of autocorrelation (use clustered or Driscoll-Kraay standard errors).")
else:
    print("No evidence of autocorrelation.")

In [ ]:
### 3. Cross-Sectional Dependence Test (Pesaran's test)
# Pesaran's test for cross-sectional dependence

# Function to compute Pesaran's Cross-Sectional Dependence (CD) test
def pesaran_cd(residuals):
    """
    Perform Pesaran's test for cross-sectional dependence on residuals.
    
    Parameters:
    residuals (pd.DataFrame or np.ndarray): Residuals from a panel model, where rows represent time periods and columns represent entities.
    
    Returns:
    stat (float): Pesaran's CD test statistic.
    p-value (float): p-value for the test.
    """
    # Check for missing values in residuals
    if residuals.isnull().any().any():
        print("Warning: Missing values found in residuals. These will be dropped.")
        residuals = residuals.dropna()  # Drop rows with missing values

    # Check for constant columns (entities with no variation in residuals)
    constant_columns = residuals.columns[residuals.nunique() == 1]
    if not constant_columns.empty:
        print(f"Warning: Constant residuals detected for entities: {constant_columns}. These will be dropped.")
        residuals = residuals.drop(columns=constant_columns)

    T, N = residuals.shape  # T: Time periods, N: Entities
    residuals = np.asarray(residuals)
    
    # Compute pairwise correlations of residuals across entities
    rho_matrix = np.corrcoef(residuals, rowvar=False)
    
    # Extract the upper triangle (off-diagonal elements) from the correlation matrix
    upper_triangle_indices = np.triu_indices(N, k=1)
    rho_values = rho_matrix[upper_triangle_indices]
    
    # Compute the Pesaran CD test statistic
    pesaran_stat = np.sqrt(2 / (N * (N - 1))) * np.sum(rho_values)
    
    # Compute the p-value using the standard normal distribution
    p_value = 2 * (1 - stats.norm.cdf(np.abs(pesaran_stat)))
    
    return pesaran_stat, p_value


# Example Usage with Panel Data and Residuals

# Fit the FE model (Fixed Effects model)
model_fe = PanelOLS(endog, exog, entity_effects=True)
fe_res = model_fe.fit(cov_type='unadjusted')

# Extract the residuals from the fitted model and reshape to (T x N) format
residuals = fe_res.resids.unstack()  # Convert to DataFrame with time as rows and entities as columns

# Perform Pesaran's test for cross-sectional dependence
pesaran_stat, p_value = pesaran_cd(residuals)

# Output the test results
print(f"Pesaran's CD test statistic: {pesaran_stat}")
print(f"Pesaran's CD test p-value: {p_value}")

# Check if cross-sectional dependence is significant
if p_value < 0.05:
    print("There is evidence of cross-sectional dependence.")
else:
    print("No evidence of cross-sectional dependence.")

### IV for total 

In [14]:
endog = df_bartik['EPS_GDP']
exog_var = df_bartik[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'Bartik_instrument']]
exog = sm.add_constant(exog_var)
model_ba = PanelOLS(endog, exog, entity_effects=True, time_effects=True) 
ba_res = model_ba.fit(cov_type='clustered')
print(ba_res)
df_bartik['pred_EPS'] = ba_res.predict().reindex(df_bartik.index).values

# Second-stage regression
endog = df_bartik['ln_GHG']
exog_var = df_bartik[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'pred_EPS']]
exog = sm.add_constant(exog_var)
model_ba = PanelOLS(endog, exog, entity_effects=True, time_effects=True)
ba_res_2 = model_ba.fit(cov_type='clustered')
print(ba_res_2)


Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.


                          PanelOLS Estimation Summary                           
Dep. Variable:                EPS_GDP   R-squared:                        0.2025
Estimator:                   PanelOLS   R-squared (Between):             -33.612
No. Observations:                1138   R-squared (Within):              -0.2700
Date:                Fri, Feb 21 2025   R-squared (Overall):             -4.3826
Time:                        10:32:47   Log-likelihood                   -615.25
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      53.741
Entities:                          46   P-value                           0.0000
Avg Obs:                       24.739   Distribution:                  F(5,1058)
Min Obs:                       1.0000                                           
Max Obs:                       30.000   F-statistic (robust):             48.183
                            

In [121]:
# Efficient f stat
# Get the coefficient and standard error of the instrument in the first stage
instrument_coef = ba_res.params['Bartik_instrument']
instrument_se = ba_res.std_errors['Bartik_instrument']

# Compute the approximate effective F-statistic
effective_f_stat = (instrument_coef / instrument_se) ** 2
print("Approximate Effective F-Statistic:", effective_f_stat)

Approximate Effective F-Statistic: 46.282276240716385


### IV for sectoral

In [138]:
# First-stage regressions

# Building
endog = df_bartik['EPS_OCDE_building']
exog_var = df_bartik[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'Bartik_instrument_building']]
exog = sm.add_constant(exog_var)
model_ba = PanelOLS(endog, exog, entity_effects=True, time_effects=True) 
ba_res_building = model_ba.fit(cov_type='clustered')
print(ba_res_building)
df_bartik['pred_EPS_building'] = ba_res_building.predict().reindex(df_bartik.index).values

# Electricity
endog = df_bartik['EPS_OCDE_elec']
exog_var = df_bartik[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'Bartik_instrument_elec']]
exog = sm.add_constant(exog_var)
model_ba = PanelOLS(endog, exog, entity_effects=True, time_effects=True)
ba_res_elec = model_ba.fit(cov_type='clustered')
print(ba_res_elec)
df_bartik['pred_EPS_elec'] = ba_res_elec.predict().reindex(df_bartik.index).values

# Industry
endog = df_bartik['EPS_OCDE_indus']
exog_var = df_bartik[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'Bartik_instrument_indus']]
exog = sm.add_constant(exog_var)
model_ba = PanelOLS(endog, exog, entity_effects=True, time_effects=True)
ba_res_indus = model_ba.fit(cov_type='clustered')
print(ba_res_indus)
df_bartik['pred_EPS_indus'] = ba_res_indus.predict().reindex(df_bartik.index).values

# Transport
endog = df_bartik['EPS_OCDE_transport']
exog_var = df_bartik[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'Bartik_instrument_transport']]
exog = sm.add_constant(exog_var)
model_ba = PanelOLS(endog, exog, entity_effects=True, time_effects=True)
ba_res_transport = model_ba.fit(cov_type='clustered')
print(ba_res_transport)
df_bartik['pred_EPS_transport'] = ba_res_transport.predict().reindex(df_bartik.index).values



# Second-stage regression
endog = df_bartik['ln_GHG']
exog_var = df_bartik[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'pred_EPS_building', 'pred_EPS_elec', 'pred_EPS_indus', 'pred_EPS_transport']]
exog = sm.add_constant(exog_var)
model_ba = PanelOLS(endog, exog, entity_effects=True, time_effects=True)
ba_res_2 = model_ba.fit(cov_type='clustered')
print(ba_res_2)



Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.


                          PanelOLS Estimation Summary                           
Dep. Variable:      EPS_OCDE_building   R-squared:                        0.1032
Estimator:                   PanelOLS   R-squared (Between):             -18.754
No. Observations:                1138   R-squared (Within):              -0.2781
Date:                Sun, Feb 16 2025   R-squared (Overall):             -2.7267
Time:                        17:18:36   Log-likelihood                    663.95
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      24.341
Entities:                          46   P-value                           0.0000
Avg Obs:                       24.739   Distribution:                  F(5,1058)
Min Obs:                       1.0000                                           
Max Obs:                       30.000   F-statistic (robust):             19.394
                            


Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.


### TETS on IV

#### the plausible exogeneity test proposed by Conley, Hansen, and Rossi in their 2012 paper, Plausibly Exogenous

In [125]:
import scipy.stats as stats
# Step 1: Estimate the structural equation (second stage) without the instrument to obtain residuals
# Using all control variables except the Bartik instrument
endog_structural = df_bartik['ln_GHG']
exog_controls = sm.add_constant(df_bartik[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth']])

# Run the structural equation regression without the Bartik instrument
model_structural = PanelOLS(endog_structural, exog_controls, entity_effects=True, time_effects=True)
structural_res = model_structural.fit(cov_type='clustered')

# Retrieve residuals from this regression as a proxy for the structural error term
df_bartik['structural_residuals'] = structural_res.resids


# Step 2: Perform the auxiliary regression with the Bartik instrument and structural residuals
# Define cleaned instrument and exogenous variables for auxiliary regression
endog_aux = df_bartik['Bartik_instrument']
exog_aux = sm.add_constant(df_bartik['structural_residuals'])

# Perform the auxiliary regression without NaNs/Infs
aux_model = PanelOLS(endog_aux, exog_aux, entity_effects=True, time_effects=True)
aux_res = aux_model.fit(cov_type='clustered')

# Retrieve coefficient and standard error for structural_residuals
approx_correlation = aux_res.params['structural_residuals']
approx_correlation_std = aux_res.std_errors['structural_residuals']  # Standard error

# Calculate the t-statistic
t_stat = approx_correlation / approx_correlation_std
significance_level = 0.05  # 5% significance level for two-tailed test



# Calculate p-value for the t-test (two-tailed)
from scipy.stats import t

# Determine degrees of freedom as number of observations minus parameters estimated
degrees_of_freedom = aux_res.nobs - aux_res.df_model - 1
p_value = 2 * (1 - t.cdf(abs(t_stat), df=degrees_of_freedom))

# Two-tailed critical value for a large sample at 5% significance
critical_value = stats.t.ppf(1 - significance_level / 2, degrees_of_freedom)

# Output the approximate correlation and significance test results
print(f"Approximate correlation between Bartik instrument and structural error: {approx_correlation}")
print(f"Standard error of approximate correlation: {approx_correlation_std}")
print(f"T-statistic for correlation significance: {t_stat}")
print(f"P-value for the t-test: {p_value}")
print(f"df: {degrees_of_freedom}")
print(f"cv: {critical_value}")

# Check for statistical significance
if abs(t_stat) > critical_value:
    print("The result is statistically significant, suggesting potential endogeneity of the instrument.")
else:
    print("The result is not statistically significant, supporting the exogeneity of the instrument.")



Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.


Approximate correlation between Bartik instrument and structural error: 8.147393535454462
Standard error of approximate correlation: 2.171880882019721
T-statistic for correlation significance: 3.751307727281004
P-value for the t-test: 0.00018543261555836743
df: 1061
cv: 1.9622023763656313
The result is statistically significant, suggesting potential endogeneity of the instrument.


In [152]:
# Define plausible bounds for the instrument's potential correlation with the error term
# For example, we test values in a range, say [-0.05, -0.04, ..., 0.04, 0.05]
bounds = np.linspace(-0.2, 0.2, 5)  # Adjust this range based on your domain knowledge

# Collect results for each bound to analyze the sensitivity
sensitivity_results = []

for delta in bounds:
    # First Stage: Estimate EPS_OCDE as a function of exogenous variables and Bartik instrument
    endog_1 = df_bartik['EPS_OCDE']
    exog_1 = sm.add_constant(df_bartik[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'Bartik_instrument']])
    model_stage1 = PanelOLS(endog_1, exog_1, entity_effects=True, time_effects=True)
    stage1_res = model_stage1.fit(cov_type='clustered')
    
    # Predicted values from the first stage
    # Obtain predictions for the same index in `df_bartik`
    predicted_values_cor1 = stage1_res.predict()
    # Align predictions to `df_bartik` using reindex
    df_bartik['pred_EPS_cor1'] = predicted_values_cor1.reindex(df_bartik.index).values


    # Second Stage: Adjusting the outcome variable by adding the delta term
    # Here, ln_GHG is regressed on the predicted EPS with the delta adjustment
    endog_2 = df_bartik['ln_GHG'] - delta * df_bartik['Bartik_instrument']
    exog_2 = sm.add_constant(df_bartik[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'pred_EPS_cor1']])
    
    # Run the second stage regression
    model_stage2 = PanelOLS(endog_2, exog_2, entity_effects=True, time_effects=True)
    stage2_res = model_stage2.fit(cov_type='clustered')
    
    # Collect results
    sensitivity_results.append((delta, stage2_res.params, stage2_res.tstats))

# Display results for each delta bound
for delta, params, tstats in sensitivity_results:
    print(f"Delta: {delta}")
    print("Parameters:")
    print(params)
    print("T-Statistics:")
    print(tstats)
    print("\n" + "-"*30 + "\n")



Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.


Delta: -0.2
Parameters:
const            618.919252
ln_GDP           -55.260599
ln_GDP_2           2.266514
ln_POP           -15.755233
urban_growth       0.239930
pred_EPS_cor1     -8.508824
Name: parameter, dtype: float64
T-Statistics:
const            164.816242
ln_GDP          -153.527799
ln_GDP_2         161.985215
ln_POP          -161.721694
urban_growth      34.084598
pred_EPS_cor1   -194.459351
Name: tstat, dtype: float64

------------------------------

Delta: -0.1
Parameters:
const            314.187673
ln_GDP           -28.201187
ln_GDP_2           1.167384
ln_POP            -7.693938
urban_growth       0.103184
pred_EPS_cor1     -4.336509
Name: parameter, dtype: float64
T-Statistics:
const            83.667185
ln_GDP          -78.349970
ln_GDP_2         83.431643
ln_POP          -78.975456
urban_growth     14.658350
pred_EPS_cor1   -99.105904
Name: tstat, dtype: float64

------------------------------

Delta: 0.0
Parameters:
const            9.456093
ln_GDP          -1.1417


Inputs contain missing values. Dropping rows with missing observations.


#### IV Tests from Borusyak et al 2023

In [171]:
# Ensure Year is numeric
df_bartik['Year'] = pd.to_numeric(df_bartik['Year'], errors='coerce')

# Define period 1 (1991–2000)
period_1 = df_bartik[(df_bartik['Year'] >= 1991) & (df_bartik['Year'] <= 2000)]
    
# First stage
endog_1st = period_1['EPS_BOD_100var']
exog_var_1st = period_1[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'Bartik_instrument']]
exog_1st = sm.add_constant(exog_var_1st)
model_1st = PanelOLS(endog_1st, exog_1st, entity_effects=True, time_effects=True)
res_1st = model_1st.fit(cov_type='clustered')

period_1.loc[:, 'predicted_EPS'] = res_1st.predict().values

# Second stage
endog_2nd = period_1['ln_GHG']
exog_var_2nd = period_1[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'predicted_EPS']]
exog_2nd = sm.add_constant(exog_var_2nd)
model_2nd = PanelOLS(endog_2nd, exog_2nd, entity_effects=True, time_effects=True)
res_2nd = model_2nd.fit(cov_type='clustered')

print("\nResults for Period 1 (1991_2000):")
print(res_2nd)





Results for Period 1 (1991_2000):
                          PanelOLS Estimation Summary                           
Dep. Variable:                 ln_GHG   R-squared:                        0.4199
Estimator:                   PanelOLS   R-squared (Between):              0.2314
No. Observations:                 293   R-squared (Within):              -0.3354
Date:                Sun, Feb 16 2025   R-squared (Overall):              0.1718
Time:                        18:38:59   Log-likelihood                    507.95
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      34.308
Entities:                          42   P-value                           0.0000
Avg Obs:                       6.9762   Distribution:                   F(5,237)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [172]:
# Ensure Year is numeric
df_bartik['Year'] = pd.to_numeric(df_bartik['Year'], errors='coerce')

# Define period 1 (1991–2000)
period_2 = df_bartik[(df_bartik['Year'] > 2000) & (df_bartik['Year'] <= 2010)]
    
# First stage
endog_1st = period_2['EPS_BOD_100var']
exog_var_1st = period_2[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'Bartik_instrument']]
exog_1st = sm.add_constant(exog_var_1st)
model_1st = PanelOLS(endog_1st, exog_1st, entity_effects=True, time_effects=True)
res_1st = model_1st.fit(cov_type='clustered')

period_2.loc[:, 'predicted_EPS'] = res_1st.predict().values

# Second stage
endog_2nd = period_2['ln_GHG']
exog_var_2nd = period_2[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'predicted_EPS']]
exog_2nd = sm.add_constant(exog_var_2nd)
model_2nd = PanelOLS(endog_2nd, exog_2nd, entity_effects=True, time_effects=True)
res_2nd = model_2nd.fit(cov_type='clustered')

print("\nResults for Period 2 (2001_2010):")
print(res_2nd)





Results for Period 2 (2001_2010):
                          PanelOLS Estimation Summary                           
Dep. Variable:                 ln_GHG   R-squared:                        0.4327
Estimator:                   PanelOLS   R-squared (Between):              0.6872
No. Observations:                 416   R-squared (Within):              -0.7985
Date:                Sun, Feb 16 2025   R-squared (Overall):              0.6919
Time:                        18:39:26   Log-likelihood                    748.84
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      54.621
Entities:                          44   P-value                           0.0000
Avg Obs:                       9.4545   Distribution:                   F(5,358)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [173]:
# Ensure Year is numeric
df_bartik['Year'] = pd.to_numeric(df_bartik['Year'], errors='coerce')

# Define period 1 (1991–2000)
period_3 = df_bartik[(df_bartik['Year'] > 2010) & (df_bartik['Year'] <= 2020)]
    
# First stage
endog_1st = period_3['EPS_BOD_100var']
exog_var_1st = period_3[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'Bartik_instrument']]
exog_1st = sm.add_constant(exog_var_1st)
model_1st = PanelOLS(endog_1st, exog_1st, entity_effects=True, time_effects=True)
res_1st = model_1st.fit(cov_type='clustered')

period_3.loc[:, 'predicted_EPS'] = res_1st.predict().values

# Second stage
endog_2nd = period_3['ln_GHG']
exog_var_2nd = period_3[['ln_GDP', 'ln_POP', 'urban_growth', 'predicted_EPS']]
exog_2nd = sm.add_constant(exog_var_2nd)
model_2nd = PanelOLS(endog_2nd, exog_2nd, entity_effects=True, time_effects=True)
res_2nd = model_2nd.fit(cov_type='clustered')

print("\nResults for Period 3 (2011_2020):")
print(res_2nd)





Results for Period 3 (2011_2020):
                          PanelOLS Estimation Summary                           
Dep. Variable:                 ln_GHG   R-squared:                        0.2166
Estimator:                   PanelOLS   R-squared (Between):              0.5908
No. Observations:                 429   R-squared (Within):              -0.2298
Date:                Sun, Feb 16 2025   R-squared (Overall):              0.5912
Time:                        18:39:43   Log-likelihood                    668.72
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      25.572
Entities:                          46   P-value                           0.0000
Avg Obs:                       9.3261   Distribution:                   F(4,370)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [174]:
# Step 3: Placebo Test - Generate placebo EPS_shift by permuting values
np.random.seed(0)  # Set seed for reproducibility
df_bartik['EPS_shift_placebo'] = np.random.permutation(df_bartik['EPS_shift'].values)

# Calculate placebo Bartik instrument
df_bartik['Bartik_instrument_placebo'] = df_bartik['agri_0_share'] * df_bartik['EPS_shift_placebo']

# Step 4: Run placebo IV regression with the placebo Bartik instrument
endog_1st_placebo = df_bartik['EPS_BOD_100var']
exog_var_1st_placebo = df_bartik[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'Bartik_instrument_placebo']]
exog_1st_placebo = sm.add_constant(exog_var_1st_placebo)
model_1st_placebo = PanelOLS(endog_1st_placebo, exog_1st_placebo, entity_effects=True, time_effects=True)
res_1st_placebo = model_1st_placebo.fit(cov_type='clustered')

# Save predicted values from the placebo first stage
predicted_placebo = res_1st_placebo.predict()
# Align predictions to `df_bartik` using reindex
df_bartik['predicted_EPS_placebo'] = predicted_placebo.reindex(df_bartik.index).values


# Second stage with placebo predicted EPS
endog_2nd_placebo = df_bartik['ln_GHG']
exog_var_2nd_placebo = df_bartik[['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'predicted_EPS_placebo']]
exog_2nd_placebo = sm.add_constant(exog_var_2nd_placebo)
model_2nd_placebo = PanelOLS(endog_2nd_placebo, exog_2nd_placebo, entity_effects=True, time_effects=True)
res_2nd_placebo = model_2nd_placebo.fit(cov_type='clustered')

print("\nResults for placebo shock")
print(res_2nd_placebo)


Inputs contain missing values. Dropping rows with missing observations.

Inputs contain missing values. Dropping rows with missing observations.



Results for placebo shock
                          PanelOLS Estimation Summary                           
Dep. Variable:                 ln_GHG   R-squared:                        0.4566
Estimator:                   PanelOLS   R-squared (Between):              0.9091
No. Observations:                1138   R-squared (Within):              -1.2444
Date:                Sun, Feb 16 2025   R-squared (Overall):              0.8853
Time:                        18:40:06   Log-likelihood                    1199.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      177.78
Entities:                          46   P-value                           0.0000
Avg Obs:                       24.739   Distribution:                  F(5,1058)
Min Obs:                       1.0000                                           
Max Obs:                       30.000   F-statistic (robust):             102.50
 

#### Borusiak Normmal models

In [115]:
# Ensure Year is numeric
borusiak = modGDP.copy()
borusiak['Year'] = pd.to_numeric(borusiak['Year'], errors='coerce')

# Define period 1 (1991–2000)
period_1 = borusiak[(borusiak['Year'] >= 1991) & (borusiak['Year'] <= 2000)]
    
# reg
endog1 = period_1['ln_GHG_per_cap']
exog_var1 = period_1[['ln_GDP_per_cap', 'ln_GDP_per_cap_2', 'POP_growth', 'urban_growth', 'EPS_GDP']]
exog1 = sm.add_constant(exog_var1)
model1 = PanelOLS(endog1, exog1, entity_effects=True, time_effects=True)
res1 = model1.fit(cov_type='clustered')

print("\nResults for Period 1 (1991_2000):")
print(res1)

# Define period 2 (2001–2010)
period_2 = borusiak[(borusiak['Year'] > 2000) & (borusiak['Year'] <= 2010)]
    
# reg
endog2 = period_2['ln_GHG_per_cap']
exog_var2 = period_2[['ln_GDP_per_cap', 'ln_GDP_per_cap_2', 'POP_growth', 'urban_growth', 'EPS_GDP']]
exog2 = sm.add_constant(exog_var2)
model2 = PanelOLS(endog2, exog2, entity_effects=True, time_effects=True)
res2 = model2.fit(cov_type='clustered')

print("\nResults for Period 2 (2001_2010):")
print(res2)


# Define period 3 (2011–2020)
period_3 = borusiak[(borusiak['Year'] > 2010) & (borusiak['Year'] <= 2020)]
    
# reg
endog3 = period_3['ln_GHG_per_cap']
exog_var3 = period_3[['ln_GDP_per_cap', 'ln_GDP_per_cap_2', 'POP_growth', 'urban_growth', 'EPS_GDP']]
exog3 = sm.add_constant(exog_var3)
model3 = PanelOLS(endog3, exog3, entity_effects=True, time_effects=True)
res3 = model3.fit(cov_type='clustered')

print("\nResults for Period 3 (2011_2020):")
print(res3)


Inputs contain missing values. Dropping rows with missing observations.



Results for Period 1 (1991_2000):
                          PanelOLS Estimation Summary                           
Dep. Variable:         ln_GHG_per_cap   R-squared:                        0.2144
Estimator:                   PanelOLS   R-squared (Between):              0.4269
No. Observations:                 292   R-squared (Within):              -0.2914
Date:                Sun, Feb 16 2025   R-squared (Overall):              0.4050
Time:                        16:48:13   Log-likelihood                    493.41
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      12.878
Entities:                          42   P-value                           0.0000
Avg Obs:                       6.9524   Distribution:                   F(5,236)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             

In [ ]:
modGDP['ln_GDP_2'] = modGDP['ln_GDP']**2
# Ensure Year is numeric
borusiak = modGDP.copy()
borusiak['Year'] = pd.to_numeric(borusiak['Year'], errors='coerce')


# Define period 2 (2001–2010)
period_2 = borusiak[(borusiak['Year'] > 2000)]
    
# reg
endog2 = period_2['ln_GHG']
exog_var2 = period_2[['ln_GDP', 'ln_GDP_2','ln_POP', 'urban_growth', 'EPS_GDP']]
exog2 = sm.add_constant(exog_var2)
model2 = PanelOLS(endog2, exog2, entity_effects=True, time_effects=True)
res2 = model2.fit(cov_type='clustered')

print("\nResults for Period 2 (2001_2010):")
print(res2)


In [119]:
## Period GHG per GDP 

# Ensure Year is numeric
borusiak = modBOD.copy()
borusiak['Year'] = pd.to_numeric(borusiak['Year'], errors='coerce')
borusiak['GHG_per_GDP'] = borusiak['GHG'] / borusiak['GDP']*1000
borusiak['ln_GHG_per_GDP'] = np.log(borusiak['GHG_per_GDP'])

# Define period 1 (1991–2000)
period_1 = borusiak[(borusiak['Year'] >= 1991) & (borusiak['Year'] <= 2000)]
    
# reg
endog1 = period_1['ln_GHG_per_GDP']
exog_var1 = period_1[['GDP_growth', 'GDP_growth_2','POP_growth', 'urban_growth', 'EPS_BOD_100var']]
exog1 = sm.add_constant(exog_var1)
model1 = PanelOLS(endog1, exog1, entity_effects=True, time_effects=True)
res1 = model1.fit(cov_type='clustered')

print("\nResults for Period 1 (1991_2000):")
print(res1)

# Define period 2 (2001–2010)
period_2 = borusiak[(borusiak['Year'] > 2000) & (borusiak['Year'] <= 2010)]
    
# reg
endog2 = period_2['ln_GHG_per_GDP']
exog_var2 = period_2[['GDP_growth', 'GDP_growth_2','POP_growth', 'urban_growth', 'EPS_BOD_100var']]
exog2 = sm.add_constant(exog_var2)
model2 = PanelOLS(endog2, exog2, entity_effects=True, time_effects=True)
res2 = model2.fit(cov_type='clustered')

print("\nResults for Period 2 (2001_2010):")
print(res2)


# Define period 3 (2011–2020)
period_3 = borusiak[(borusiak['Year'] > 2010) & (borusiak['Year'] <= 2020)]
    
# reg
endog3 = period_3['ln_GHG_per_GDP']
exog_var3 = period_3[['GDP_growth', 'GDP_growth_2', 'POP_growth', 'urban_growth', 'EPS_BOD_100var']]
exog3 = sm.add_constant(exog_var3)
model3 = PanelOLS(endog3, exog3, entity_effects=True, time_effects=True)
res3 = model3.fit(cov_type='clustered')

print("\nResults for Period 3 (2011_2020):")
print(res3)


Inputs contain missing values. Dropping rows with missing observations.



Results for Period 1 (1991_2000):
                          PanelOLS Estimation Summary                           
Dep. Variable:         ln_GHG_per_GDP   R-squared:                        0.0730
Estimator:                   PanelOLS   R-squared (Between):              0.0066
No. Observations:                 284   R-squared (Within):               0.0949
Date:                Sun, Feb 16 2025   R-squared (Overall):              0.0446
Time:                        16:53:47   Log-likelihood                    460.32
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3.6044
Entities:                          41   P-value                           0.0037
Avg Obs:                       6.9268   Distribution:                   F(5,229)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             

## Fixed effects Model by type

### BOD

#### Total, Sector, Without elec

In [386]:
#modBOD_ETS_E = modBOD[modBOD['ETS'] == 1]

exog_var = [
    'ln_GDP',
    'ln_GDP_2',
    'ln_POP',
    # 'GDP_growth',
    # 'GDP_growth_2', 
    # 'ln_GDP_per_cap',
    # 'ln_GDP_per_cap_2',
    # 'POP_growth',   
    'urban_growth',
    'EPS_BOD_100var',
    # 'ln_EPS_BOD_100var',
    # 'EPS_BOD_100var_building',
    # 'EPS_BOD_100var_elec',
    # 'EPS_BOD_100var_indus',
    # 'EPS_BOD_100var_transport'
    # 'ln_EPS_BOD_100var_building',
    # 'ln_EPS_BOD_100var_elec',
    # 'ln_EPS_BOD_100var_indus',
    # 'ln_EPS_BOD_100var_transport'
    # 'tr_EPS_BOD_100var_building',
    # 'tr_EPS_BOD_100var_elec',
    # 'tr_EPS_BOD_100var_indus',
    # 'tr_EPS_BOD_100var_transport'
    # 'EPS_BOD_100var_building_lag',
    # 'EPS_BOD_100var_elec_lag',
    # 'EPS_BOD_100var_indus_lag',
    # 'EPS_BOD_100var_transport_lag',
    # 'EPS_BOD_100var_building_mbi',
    # 'EPS_BOD_100var_building_nmbi', 
    # 'EPS_BOD_100var_elec_mbi',
    # 'EPS_BOD_100var_elec_nmbi',
    # 'EPS_BOD_100var_indus_mbi',
    # 'EPS_BOD_100var_indus_nmbi',
    # 'EPS_BOD_100var_transport_mbi',
    # 'EPS_BOD_100var_transport_nmbi', 
    # 'EPS_BOD_100var_mbi_mbi',
    # 'EPS_BOD_100var_nmbi_nmbi',    
]
exog = sm.add_constant(modBOD[exog_var])
endog= modBOD['ln_building_GHG']
model_fe = PanelOLS(endog, exog, entity_effects = True, time_effects = True) 
fe_res = model_fe.fit(cov_type='clustered') 
print(fe_res)


Inputs contain missing values. Dropping rows with missing observations.


                          PanelOLS Estimation Summary                           
Dep. Variable:        ln_building_GHG   R-squared:                        0.1671
Estimator:                   PanelOLS   R-squared (Between):              0.8650
No. Observations:                1124   R-squared (Within):              -0.1697
Date:                Mon, Feb 17 2025   R-squared (Overall):              0.8560
Time:                        13:40:18   Log-likelihood                    352.17
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      41.889
Entities:                          45   P-value                           0.0000
Avg Obs:                       24.978   Distribution:                  F(5,1044)
Min Obs:                       1.0000                                           
Max Obs:                       31.000   F-statistic (robust):             18.538
                            

#### GHG/GDP

In [253]:
modBOD['GHG_per_GDP'] = modBOD['GHG'] / modBOD['GDP']*1000
modBOD['ln_GHG_per_GDP'] = np.log(modBOD['GHG_per_GDP'])
modBOD['transport*elec'] = modBOD['EPS_BOD_100var_transport'] * modBOD['EPS_BOD_100var_elec']

# Liste des variables exogènes
exog_var = [
    'ln_POP',
    'GDP_growth',
    'GDP_growth_2', 
    'urban_growth',
    # 'EPS_BOD_100var'
    'EPS_BOD_100var_building', 'EPS_BOD_100var_elec', 'EPS_BOD_100var_indus', 'EPS_BOD_100var_transport'
]

# Ajouter une constante aux variables exogènes
exog = sm.add_constant(modBOD[exog_var])

# Mettre à jour la variable endogène
endog = modBOD['ln_GHG_per_GDP']

# Estimer le modèle avec effets fixes
model_fe = PanelOLS(endog, exog, entity_effects=True, time_effects=True)
fe_res = model_fe.fit(cov_type='clustered')

print(fe_res)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

Inputs contain missing values. Dropping rows with missing observations.


                          PanelOLS Estimation Summary                           
Dep. Variable:         ln_GHG_per_GDP   R-squared:                        0.1927
Estimator:                   PanelOLS   R-squared (Between):             -0.7053
No. Observations:                1098   R-squared (Within):              -0.0946
Date:                Mon, Feb 17 2025   R-squared (Overall):             -0.9684
Time:                        10:29:11   Log-likelihood                    1064.0
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      30.318
Entities:                          45   P-value                           0.0000
Avg Obs:                       24.400   Distribution:                  F(8,1016)
Min Obs:                       1.0000                                           
Max Obs:                       30.000   F-statistic (robust):             20.745
                            

#### With interaction

In [21]:
modBOD_100var = (modBOD
          .assign(building_elec=lambda df: df['EPS_BOD_100var_building'] * df['EPS_BOD_100var_elec'])
          .assign(transport_indus=lambda df: df['EPS_BOD_100var_transport'] * df['EPS_BOD_100var_indus'])
          .assign(elec_indus=lambda df: df['EPS_BOD_100var_elec'] * df['EPS_BOD_100var_indus'])
          .assign(transport_building=lambda df: df['EPS_BOD_100var_transport'] * df['EPS_BOD_100var_building'])
          .assign(transport_elec=lambda df: df['EPS_BOD_100var_transport'] * df['EPS_BOD_100var_elec'])
          .assign(indus_building=lambda df: df['EPS_BOD_100var_indus'] * df['EPS_BOD_100var_building'])
          .assign(eps_ets=lambda df: df['EPS_BOD_100var'] * df['ETS'])
          .assign(eps_ets_b=lambda df: df['EPS_BOD_100var_building'] * df['ETS_B'])
          .assign(eps_ets_i=lambda df: df['EPS_BOD_100var_indus'] * df['ETS_I'])
          .assign(eps_ets_e=lambda df: df['EPS_BOD_100var_elec'] * df['ETS_E'])
          .assign(eps_ets_t=lambda df: df['EPS_BOD_100var_transport'] * df['ETS_T'])
         )

# Liste des variables exogènes incluant l'interaction
exog_vars_with_interaction = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth',  
                                'EPS_BOD_100var_indus','EPS_BOD_100var_transport', 'EPS_BOD_100var_building', 'EPS_BOD_100var_elec',
                                # 'building_elec', 'transport_indus', 'indus_building',                               
                                'elec_indus', 'transport_elec', 'transport_building',
                                # 'ETS_B', 'ETS_E',
                                # 'ETS_I',  'ETS_T',  'eps_ets_t'
                                # 'eps_ets_b', 'eps_ets_e', 'eps_ets_i',
                                ]


# Construire les variables exogènes et endogènes
exog_with_interaction = sm.add_constant(modBOD_100var[exog_vars_with_interaction])
endog = modBOD_100var['ln_GHG']

# Modèle avec effets fixes
model_fe_with_interaction = PanelOLS(endog, exog_with_interaction, entity_effects=True, time_effects=True) 
fe_res_with_interaction = model_fe_with_interaction.fit(cov_type='clustered') 
print(fe_res_with_interaction)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 ln_GHG   R-squared:                        0.5498
Estimator:                   PanelOLS   R-squared (Between):              0.9320
No. Observations:                1152   R-squared (Within):               0.1170
Date:                Fri, Feb 21 2025   R-squared (Overall):              0.9197
Time:                        16:19:18   Log-likelihood                    1280.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      118.24
Entities:                          46   P-value                           0.0000
Avg Obs:                       25.043   Distribution:                 F(11,1065)
Min Obs:                       1.0000                                           
Max Obs:                       31.000   F-statistic (robust):             79.676
                            

##### tests with interaction 

In [321]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Step 1: Sélectionner les variables d'intérêt, y compris 'Year'
variables_of_interest = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth',  
                                'EPS_BOD_100var_indus','EPS_BOD_100var_transport', 'EPS_BOD_100var_building', 'EPS_BOD_100var_elec',
                                #                               
                                # 'building_elec', 'transport_indus', 'indus_building', 'elec_indus', 'transport_elec', 'transport_building',
                                # 'ETS_B', 'ETS_E',
                                # 'ETS_I',  'ETS_T',  'eps_ets_t'
                                'eps_ets_b', 'eps_ets_e', 'eps_ets_i',
                                ]

# Step 2: Nettoyer les données
# Remplacer les valeurs infinies par NaN, puis supprimer les lignes avec des NaN dans les variables d'intérêt
mod_cleaned = (
    modBOD_100var[variables_of_interest]  # On garde uniquement les colonnes d'intérêt
    .replace([np.inf, -np.inf], np.nan)  # Remplacer les infinis par NaN
    .dropna()  # Supprimer les lignes avec des NaN
)

# Step 3: Ajouter une constante pour le calcul du VIF
X_with_constant = sm.add_constant(mod_cleaned)

# Step 4: Calculer le VIF pour chaque variable, y compris 'Year'
vif_data = pd.DataFrame({
    "Variable": X_with_constant.columns,
    "VIF": [variance_inflation_factor(X_with_constant.values, i) for i in range(X_with_constant.shape[1])]
})

# Step 5: Afficher les résultats du VIF
print(vif_data)

                    Variable          VIF
0                      const  3371.692521
1                     ln_GDP   223.264499
2                   ln_GDP_2   217.801860
3                     ln_POP     5.748252
4               urban_growth     1.229537
5       EPS_BOD_100var_indus    17.706469
6   EPS_BOD_100var_transport     6.448361
7    EPS_BOD_100var_building    10.387784
8        EPS_BOD_100var_elec    15.294315
9                  eps_ets_b     1.139824
10                 eps_ets_e    13.992219
11                 eps_ets_i    18.761052


In [94]:
#### avec demean

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Step 1: Define independent variables
variables_of_interest = [
                        'ln_GDP', 'ln_GDP_2', 
                         'ln_POP', 'urban_growth',  
                        'EPS_BOD_100var_indus',
                        #  'EPS_BOD_100var_transport', 'EPS_BOD_100var_building', 'EPS_BOD_100var_elec',
                        # 'building_elec', 'transport_indus', 'indus_building', 
                        'elec_indus', 'transport_elec', 'transport_building',
                        ]


# Step 3: Apply transformations for fixed effects demeaning
vif_data = (
    modBOD_100var[variables_of_interest].copy()  # Create a copy for safe transformations
    .assign( 
        **{col: lambda df, col=col: df[col] - df.groupby('iso3')[col].transform('mean') for col in variables_of_interest}

    )
    .assign( 
        **{col: lambda df, col=col: df[col] - df.groupby('Year')[col].transform('mean') for col in variables_of_interest}

    )
    # .pipe(lambda df: df.loc[:, df.std() > 1e-8])  # Drop near-zero variance columns
    .pipe(lambda df: sm.add_constant(df))  # Add constant for VIF computation
    .pipe(  # Compute VIF
        lambda df: pd.DataFrame({
            "Variable": df.columns,
            "VIF": [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
        })
    )
)


# Step 4: Display VIF results
print(vif_data)


               Variable        VIF
0                 const   1.000000
1                ln_GDP  24.762860
2              ln_GDP_2  24.816833
3                ln_POP   1.254368
4          urban_growth   1.096024
5  EPS_BOD_100var_indus   6.218561
6            elec_indus  10.330772
7        transport_elec   8.013652
8    transport_building   5.228377


In [291]:
### Computation of total average effects
# Extraire les coefficients
coefficients = fe_res_with_interaction.params

# Calculer l'effet total
# Suppose coefficients are extracted as:
# beta_elec = coefficients['EPS_BOD_100var_indus']
beta_transport_elec = coefficients['transport_elec']
# beta_building = coefficients['EPS_BOD_100var_building']
beta_indus = coefficients['EPS_BOD_100var_indus']
# beta_transport = coefficients['EPS_BOD_100var_transport']
beta_elec_indus = coefficients['elec_indus']
beta_transport_building = coefficients['transport_building']

# Calculer les contributions individuelles

modBOD = (modBOD
        #   .assign(effect_elec=lambda df: beta_elec * df['EPS_BOD_100var_elec'])
          .assign(effect_transport_elec=lambda df: beta_transport_elec * (df['EPS_BOD_100var_elec'] * df['EPS_BOD_100var_transport']))
        #   .assign(effect_building=lambda df: beta_building * df['EPS_BOD_100var_building'])
          .assign(effect_indus=lambda df: beta_indus * df['EPS_BOD_100var_indus'])
        #   .assign(effect_transport=lambda df: beta_transport * df['EPS_BOD_100var_transport'])
          .assign(effect_elec_indus=lambda df: beta_elec_indus * (df['EPS_BOD_100var_elec'] * df['EPS_BOD_100var_indus']))
          .assign(effect_transport_building=lambda df: beta_transport_building * (df['EPS_BOD_100var_building'] * df['EPS_BOD_100var_transport']))
         )


# Comparer les contributions
# average_effect_elec = modBOD['effect_elec'].mean()
# average_effect_building = modBOD['effect_building'].mean()
average_effect_indus = modBOD['effect_indus'].mean()
# average_effect_transport = modBOD['effect_transport'].mean()
average_effect_transport_elec = modBOD['effect_transport_elec'].mean()
average_effect_elec_indus = modBOD['effect_elec_indus'].mean()
average_effect_transport_building = modBOD['effect_transport_building'].mean()

# print("Average Effect of EPS_BOD_100var_elec:", average_effect_elec)
# print("Average Effect of EPS_BOD_100var_building:", average_effect_building)
# print("Average Effect of EPS_BOD_100var_transport:", average_effect_transport)
print("Average Effect of EPS_BOD_100var_indus:", average_effect_indus)
print("Average Effect of transport_elec:", average_effect_transport_elec)
print("Average Effect of elec_indus:", average_effect_elec_indus)
print("Average Effect of transport_building:", average_effect_transport_building)

Average Effect of EPS_BOD_100var_indus: -0.07251317503787871
Average Effect of transport_elec: -0.08002833501042846
Average Effect of elec_indus: 0.02726832347313082
Average Effect of transport_building: 0.03583771398530169


### OCDE

#### Total, Sector, Without elec

In [126]:

exog_var = [
    'ln_GDP',
    'ln_GDP_2',
    'ln_POP', 
    # 'ln_GDP_per_cap',
    # 'ln_GDP_per_cap_2',
    # 'POP_growth',   
    'urban_growth',
    'EPS_OCDE',
    # 'ln_EPS_OCDE',
    # 'EPS_OCDE_building',
    # 'EPS_OCDE_elec',
    # 'EPS_OCDE_indus',
    # 'EPS_OCDE_transport'
    # 'ln_EPS_OCDE_building',
    # 'ln_EPS_OCDE_elec',
    # 'ln_EPS_OCDE_indus',
    # 'ln_EPS_OCDE_transport'
    # 'EPS_OCDE_building_lag',
    # 'EPS_OCDE_elec_lag',
    # 'EPS_OCDE_indus_lag',
    # 'EPS_OCDE_transport_lag',
    # 'EPS_OCDE_building_mbi',
    # 'EPS_OCDE_building_nmbi', 
    # 'EPS_OCDE_elec_mbi',
    # 'EPS_OCDE_elec_nmbi',
    # 'EPS_OCDE_indus_mbi',
    # 'EPS_OCDE_indus_nmbi',
    # 'EPS_OCDE_transport_mbi',
    # 'EPS_OCDE_transport_nmbi', 
    # 'EPS_OCDE_mbi_mbi',
    # 'EPS_OCDE_nmbi_nmbi',
       
]


exog = sm.add_constant(modOCDE[exog_var])
endog= modOCDE['GHG']
model_fe = PanelOLS(endog, exog, entity_effects = True, time_effects = True) 
fe_res = model_fe.fit(cov_type='clustered')

print(fe_res)


                          PanelOLS Estimation Summary                           
Dep. Variable:                    GHG   R-squared:                        0.4592
Estimator:                   PanelOLS   R-squared (Between):              0.5349
No. Observations:                1152   R-squared (Within):               0.1701
Date:                Wed, Feb 19 2025   R-squared (Overall):              0.5405
Time:                        19:13:51   Log-likelihood                -1.371e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      181.88
Entities:                          46   P-value                           0.0000
Avg Obs:                       25.043   Distribution:                  F(5,1071)
Min Obs:                       1.0000                                           
Max Obs:                       31.000   F-statistic (robust):             89.361
                            

#### With interactions

In [20]:
modOCDE = (modOCDE
          .assign(building_elec=lambda df: df['EPS_OCDE_building'] * df['EPS_OCDE_elec'])
          .assign(transport_indus=lambda df: df['EPS_OCDE_transport'] * df['EPS_OCDE_indus'])
          .assign(elec_indus=lambda df: df['EPS_OCDE_elec'] * df['EPS_OCDE_indus'])
          .assign(transport_building=lambda df: df['EPS_OCDE_transport'] * df['EPS_OCDE_building'])
          .assign(transport_elec=lambda df: df['EPS_OCDE_transport'] * df['EPS_OCDE_elec'])
          .assign(indus_building=lambda df: df['EPS_OCDE_indus'] * df['EPS_OCDE_building'])
          .assign(eps_ets=lambda df: df['EPS_OCDE'] * df['ETS'])
          .assign(eps_ets_b=lambda df: df['EPS_OCDE_building'] * df['ETS_B'])
          .assign(eps_ets_i=lambda df: df['EPS_OCDE_indus'] * df['ETS_I'])
          .assign(eps_ets_e=lambda df: df['EPS_OCDE_elec'] * df['ETS_E'])
          .assign(eps_ets_t=lambda df: df['EPS_OCDE_transport'] * df['ETS_T'])
         )

# Liste des variables exogènes incluant l'interaction
exog_vars_with_interaction = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth',  
                                'EPS_OCDE_indus','EPS_OCDE_transport', 'EPS_OCDE_building', 'EPS_OCDE_elec',
                                #'building_elec', 'transport_indus', 'indus_building',                            
                                'elec_indus', 'transport_elec', 'transport_building',
                                # 'ETS_B', 'ETS_E',
                                # 'ETS_I',  'ETS_T',  'eps_ets_t'
                                # 'eps_ets_b', 'eps_ets_e', 'eps_ets_i',
                                # 'eps_ets'
                                ]


# Construire les variables exogènes et endogènes
exog_with_interaction = sm.add_constant(modOCDE[exog_vars_with_interaction])
endog = modOCDE['ln_GHG']

# Modèle avec effets fixes
model_fe_with_interaction = PanelOLS(endog, exog_with_interaction, entity_effects=True, time_effects=True) 
fe_res_with_interaction = model_fe_with_interaction.fit(cov_type='clustered') 
print(fe_res_with_interaction)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 ln_GHG   R-squared:                        0.5791
Estimator:                   PanelOLS   R-squared (Between):              0.9358
No. Observations:                1152   R-squared (Within):               0.4762
Date:                Fri, Feb 21 2025   R-squared (Overall):              0.9262
Time:                        16:18:16   Log-likelihood                    1319.0
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      133.21
Entities:                          46   P-value                           0.0000
Avg Obs:                       25.043   Distribution:                 F(11,1065)
Min Obs:                       1.0000                                           
Max Obs:                       31.000   F-statistic (robust):             97.227
                            

##### Tests with interaction 

In [19]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Step 1: Sélectionner les variables d'intérêt, y compris 'Year'
variables_of_interest = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth',  
                                'EPS_OCDE_indus','EPS_OCDE_transport', 'EPS_OCDE_building', 'EPS_OCDE_elec',
                                #  'building_elec', 'transport_indus', 'indus_building',                               
                                'elec_indus', 'transport_elec', 'transport_building',
                                # 'ETS_B', 'ETS_E',
                                # 'ETS_I',  'ETS_T',  'eps_ets_t'
                                # 'eps_ets_b', 'eps_ets_e', 'eps_ets_i',
                                ]

# Step 2: Nettoyer les données
# Remplacer les valeurs infinies par NaN, puis supprimer les lignes avec des NaN dans les variables d'intérêt
mod_cleaned = (
    modOCDE[variables_of_interest]  # On garde uniquement les colonnes d'intérêt
    .replace([np.inf, -np.inf], np.nan)  # Remplacer les infinis par NaN
    .dropna()  # Supprimer les lignes avec des NaN
)

# Step 3: Ajouter une constante pour le calcul du VIF
X_with_constant = sm.add_constant(mod_cleaned)

# Step 4: Calculer le VIF pour chaque variable, y compris 'Year'
vif_data = pd.DataFrame({
    "Variable": X_with_constant.columns,
    "VIF": [variance_inflation_factor(X_with_constant.values, i) for i in range(X_with_constant.shape[1])]
})

# Step 5: Afficher les résultats du VIF
print(vif_data)

KeyError: "['elec_indus', 'transport_elec', 'transport_building'] not in index"

In [97]:
#### avec demean

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Step 1: Define independent variables
variables_of_interest = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth',  
                                'EPS_OCDE_indus',
                                #   'EPS_OCDE_transport', 'EPS_OCDE_building', 'EPS_OCDE_elec', 'building_elec', 'transport_indus', 'indus_building',                            
                                'elec_indus', 'transport_elec', 'transport_building',
                                # 'ETS_B', 'ETS_E',
                                # 'ETS_I',  'ETS_T',  'eps_ets_t'
                                # 'eps_ets_b', 'eps_ets_e', 'eps_ets_i',
                                ]


# Step 3: Apply transformations for fixed effects demeaning
vif_data = (
    modOCDE[variables_of_interest].copy()  # Create a copy for safe transformations
    .assign( 
        **{col: lambda df, col=col: df[col] - df.groupby('iso3')[col].transform('mean') for col in variables_of_interest}

    )
    .assign( 
        **{col: lambda df, col=col: df[col] - df.groupby('Year')[col].transform('mean') for col in variables_of_interest}

    )
    # .pipe(lambda df: df.loc[:, df.std() > 1e-8])  # Drop near-zero variance columns
    .pipe(lambda df: sm.add_constant(df))  # Add constant for VIF computation
    .pipe(  # Compute VIF
        lambda df: pd.DataFrame({
            "Variable": df.columns,
            "VIF": [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
        })
    )
)


# Step 4: Display VIF results
print(vif_data)


             Variable        VIF
0               const   1.000000
1              ln_GDP  24.132212
2            ln_GDP_2  23.993611
3              ln_POP   1.238204
4        urban_growth   1.085660
5      EPS_OCDE_indus   3.118728
6          elec_indus   6.562772
7      transport_elec   7.154868
8  transport_building   3.751705


In [298]:
### Computation of total average effects
# Extraire les coefficients
coefficients = fe_res_with_interaction.params

# Calculer l'effet total
# Suppose coefficients are extracted as:
# beta_elec = coefficients['EPS_OCDE_indus']
beta_transport_elec = coefficients['transport_elec']
# beta_building = coefficients['EPS_OCDE_building']
beta_indus = coefficients['EPS_OCDE_indus']
# beta_transport = coefficients['EPS_OCDE_transport']
beta_elec_indus = coefficients['elec_indus']
beta_transport_building = coefficients['transport_building']

# Calculer les contributions individuelles

modOCDE = (modOCDE
        #   .assign(effect_elec=lambda df: beta_elec * df['EPS_OCDE_elec'])
          .assign(effect_transport_elec=lambda df: beta_transport_elec * (df['EPS_OCDE_elec'] * df['EPS_OCDE_transport']))
        #   .assign(effect_building=lambda df: beta_building * df['EPS_OCDE_building'])
          .assign(effect_indus=lambda df: beta_indus * df['EPS_OCDE_indus'])
        #   .assign(effect_transport=lambda df: beta_transport * df['EPS_OCDE_transport'])
          .assign(effect_elec_indus=lambda df: beta_elec_indus * (df['EPS_OCDE_elec'] * df['EPS_OCDE_indus']))
          .assign(effect_transport_building=lambda df: beta_transport_building * (df['EPS_OCDE_building'] * df['EPS_OCDE_transport']))
         )


# Comparer les contributions
# average_effect_elec = modOCDE['effect_elec'].mean()
# average_effect_building = modOCDE['effect_building'].mean()
average_effect_indus = modOCDE['effect_indus'].mean()
# average_effect_transport = modOCDE['effect_transport'].mean()
average_effect_transport_elec = modOCDE['effect_transport_elec'].mean()
average_effect_elec_indus = modOCDE['effect_elec_indus'].mean()
average_effect_transport_building = modOCDE['effect_transport_building'].mean()

# print("Average Effect of EPS_OCDE_elec:", average_effect_elec)
# print("Average Effect of EPS_OCDE_building:", average_effect_building)
# print("Average Effect of EPS_OCDE_transport:", average_effect_transport)
print("Average Effect of EPS_OCDE_indus:", average_effect_indus)
print("Average Effect of transport_elec:", average_effect_transport_elec)
print("Average Effect of elec_indus:", average_effect_elec_indus)
print("Average Effect of transport_building:", average_effect_transport_building)

Average Effect of EPS_OCDE_indus: -0.07646136637196578
Average Effect of transport_elec: -0.07527580598504512
Average Effect of elec_indus: 0.031540470006218735
Average Effect of transport_building: 0.004900555176941361


#### GHG/GDP

In [257]:

modOCDE['GHG_per_GDP'] = modOCDE['GHG'] / modOCDE['GDP']*1000
modOCDE['ln_GHG_per_GDP'] = np.log(modOCDE['GHG_per_GDP'])
modOCDE['transport*elec'] = modOCDE['EPS_OCDE_transport'] * modOCDE['EPS_OCDE_elec']

# Liste des variables exogènes
exog_var = [
    'GDP_growth',
    'GDP_growth_2',
    'ln_POP', 
    'urban_growth',
    # 'EPS_OCDE'
    'EPS_OCDE_building', 'EPS_OCDE_elec', 'EPS_OCDE_indus', 'EPS_OCDE_transport'
]

# Ajouter une constante aux variables exogènes
exog = sm.add_constant(modOCDE[exog_var])

# Mettre à jour la variable endogène
endog = modOCDE['ln_GHG_per_GDP']

# Estimer le modèle avec effets fixes
model_fe = PanelOLS(endog, exog, entity_effects=True, time_effects=True)
fe_res = model_fe.fit(cov_type='clustered')

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:         ln_GHG_per_GDP   R-squared:                        0.1900
Estimator:                   PanelOLS   R-squared (Between):             -0.4335
No. Observations:                1098   R-squared (Within):               0.1273
Date:                Mon, Feb 17 2025   R-squared (Overall):             -0.6530
Time:                        10:31:48   Log-likelihood                    1062.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      29.798
Entities:                          45   P-value                           0.0000
Avg Obs:                       24.400   Distribution:                  F(8,1016)
Min Obs:                       1.0000                                           
Max Obs:                       30.000   F-statistic (robust):             20.962
                            


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

Inputs contain missing values. Dropping rows with missing observations.


In [ ]:
print(modOCDE.head())

In [ ]:
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit()
print(re_res)

### GHG

#### Total, Sector, Without elec

In [378]:

exog_var = [
    'ln_GDP',
    'ln_GDP_2',
    'ln_POP', 
    # 'ln_GDP_per_cap',
    # 'ln_GDP_per_cap_2',
    # 'POP_growth',   
    'urban_growth',
    'EPS_GHG',
    # 'ln_EPS_GHG'
    # 'EPS_GHG_building',
    # 'EPS_GHG_elec',
    # 'EPS_GHG_indus',
    # 'EPS_GHG_transport',
    # 'ln_EPS_GHG_building',
    # 'ln_EPS_GHG_elec',
    # 'ln_EPS_GHG_indus',
    # 'ln_EPS_GHG_transport'
    # 'tr_EPS_GHG_building',
    # 'tr_EPS_GHG_elec',
    # 'tr_EPS_GHG_indus',
    # 'tr_EPS_GHG_transport'
    # 'EPS_GHG_building_lag',
    # 'EPS_GHG_elec_lag',
    # 'EPS_GHG_indus_lag',
    # 'EPS_GHG_transport_lag',
    # 'EPS_GHG_building_mbi',
    # 'EPS_GHG_building_nmbi', 
    # 'EPS_GHG_elec_mbi',
    # 'EPS_GHG_elec_nmbi',
    # 'EPS_GHG_indus_mbi',
    # 'EPS_GHG_indus_nmbi',
    # 'EPS_GHG_transport_mbi',
    # 'EPS_GHG_transport_nmbi', 
    # 'EPS_GHG_mbi_mbi',
    # 'EPS_GHG_nmbi_nmbi',    
]
exog = sm.add_constant(modGHG[exog_var])
endog= modGHG['ln_building_GHG']
model_fe = PanelOLS(endog, exog, entity_effects = True, time_effects = True) 
fe_res = model_fe.fit(cov_type='clustered') 
print(fe_res)


Inputs contain missing values. Dropping rows with missing observations.


                          PanelOLS Estimation Summary                           
Dep. Variable:        ln_building_GHG   R-squared:                        0.1600
Estimator:                   PanelOLS   R-squared (Between):              0.8546
No. Observations:                1124   R-squared (Within):              -0.3964
Date:                Mon, Feb 17 2025   R-squared (Overall):              0.8452
Time:                        13:37:52   Log-likelihood                    347.38
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      39.763
Entities:                          45   P-value                           0.0000
Avg Obs:                       24.978   Distribution:                  F(5,1044)
Min Obs:                       1.0000                                           
Max Obs:                       31.000   F-statistic (robust):             18.377
                            

#### GHG/GDP

In [264]:
modGHG['GHG_per_GDP'] = modGHG['GHG'] / modGHG['GDP']*1000
modGHG['ln_GHG_per_GDP'] = np.log(modGHG['GHG_per_GDP'])
modGHG['transport*elec'] = modGHG['EPS_GHG_transport'] * modGHG['EPS_GHG_elec']

# Liste des variables exogènes
exog_var = [
    'GDP_growth',
    'GDP_growth_2',
    'ln_POP', 
    'urban_growth',
    # 'EPS_GHG'
    'EPS_GHG_building', 'EPS_GHG_elec', 'EPS_GHG_indus', 'EPS_GHG_transport'
]

# Ajouter une constante aux variables exogènes
exog = sm.add_constant(modGHG[exog_var])

# Mettre à jour la variable endogène
endog = modGHG['ln_GHG_per_GDP']

# Estimer le modèle avec effets fixes
model_fe = PanelOLS(endog, exog, entity_effects=True, time_effects=True)
fe_res = model_fe.fit(cov_type='clustered')

print(fe_res)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

Inputs contain missing values. Dropping rows with missing observations.


                          PanelOLS Estimation Summary                           
Dep. Variable:         ln_GHG_per_GDP   R-squared:                        0.2062
Estimator:                   PanelOLS   R-squared (Between):             -0.6251
No. Observations:                1098   R-squared (Within):               0.0516
Date:                Mon, Feb 17 2025   R-squared (Overall):             -0.8602
Time:                        10:35:56   Log-likelihood                    1073.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      32.988
Entities:                          45   P-value                           0.0000
Avg Obs:                       24.400   Distribution:                  F(8,1016)
Min Obs:                       1.0000                                           
Max Obs:                       30.000   F-statistic (robust):             23.986
                            

#### With interactions

In [18]:
modGHG = (modGHG
          .assign(building_elec=lambda df: df['EPS_GHG_building'] * df['EPS_GHG_elec'])
          .assign(transport_indus=lambda df: df['EPS_GHG_transport'] * df['EPS_GHG_indus'])
          .assign(elec_indus=lambda df: df['EPS_GHG_elec'] * df['EPS_GHG_indus'])
          .assign(transport_building=lambda df: df['EPS_GHG_transport'] * df['EPS_GHG_building'])
          .assign(transport_elec=lambda df: df['EPS_GHG_transport'] * df['EPS_GHG_elec'])
          .assign(indus_building=lambda df: df['EPS_GHG_indus'] * df['EPS_GHG_building'])
          .assign(eps_ets=lambda df: df['EPS_GHG'] * df['ETS'])
          .assign(eps_ets_b=lambda df: df['EPS_GHG_building'] * df['ETS_B'])
          .assign(eps_ets_i=lambda df: df['EPS_GHG_indus'] * df['ETS_I'])
          .assign(eps_ets_e=lambda df: df['EPS_GHG_elec'] * df['ETS_E'])
          .assign(eps_ets_t=lambda df: df['EPS_GHG_transport'] * df['ETS_T'])
         )

# Liste des variables exogènes incluant l'interaction
exog_vars_with_interaction = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth',  
                                'EPS_GHG_indus','EPS_GHG_transport', 'EPS_GHG_building', 'EPS_GHG_elec',
                                'elec_indus', 'transport_elec', 'transport_building',                           
                                # 'building_elec', 'transport_indus', 'indus_building', 
                                # 'ETS_B', 'ETS_E',
                                # 'ETS_I',  'ETS_T',  'eps_ets_t'
                                # 'eps_ets_b', 'eps_ets_e', 'eps_ets_i',
                                # 'eps_ets', 'ETS'
                                ]


# Construire les variables exogènes et endogènes
exog_with_interaction = sm.add_constant(modGHG[exog_vars_with_interaction])
endog = modGHG['ln_GHG']

# Modèle avec effets fixes
model_fe_with_interaction = PanelOLS(endog, exog_with_interaction, entity_effects=True, time_effects=True) 
fe_res_with_interaction = model_fe_with_interaction.fit(cov_type='clustered') 
print(fe_res_with_interaction)


Inputs contain missing values. Dropping rows with missing observations.


                          PanelOLS Estimation Summary                           
Dep. Variable:                 ln_GHG   R-squared:                        0.5974
Estimator:                   PanelOLS   R-squared (Between):              0.9346
No. Observations:                1124   R-squared (Within):               0.4480
Date:                Fri, Feb 21 2025   R-squared (Overall):              0.9329
Time:                        16:16:34   Log-likelihood                    1306.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      140.00
Entities:                          45   P-value                           0.0000
Avg Obs:                       24.978   Distribution:                 F(11,1038)
Min Obs:                       1.0000                                           
Max Obs:                       31.000   F-statistic (robust):             119.21
                            

##### Tests on interactions

In [74]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Step 1: Sélectionner les variables d'intérêt, y compris 'Year'
variables_of_interest = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth',  
                                'EPS_GHG_indus','EPS_GHG_transport', 'EPS_GHG_building', 'EPS_GHG_elec',
                                #                               
                                # 'building_elec', 'transport_indus', 'indus_building', 'elec_indus', 'transport_elec', 'transport_building',
                                # 'ETS_B', 'ETS_E',
                                # 'ETS_I',  'ETS_T',  'eps_ets_t'
                                # 'eps_ets_b', 'eps_ets_e', 'eps_ets_i',
                                ]

# Step 2: Nettoyer les données
# Remplacer les valeurs infinies par NaN, puis supprimer les lignes avec des NaN dans les variables d'intérêt
mod_cleaned = (
    modGHG[variables_of_interest]  # On garde uniquement les colonnes d'intérêt
    .replace([np.inf, -np.inf], np.nan)  # Remplacer les infinis par NaN
    .dropna()  # Supprimer les lignes avec des NaN
)

# Step 3: Ajouter une constante pour le calcul du VIF
X_with_constant = sm.add_constant(mod_cleaned)

# Step 4: Calculer le VIF pour chaque variable, y compris 'Year'
vif_data = pd.DataFrame({
    "Variable": X_with_constant.columns,
    "VIF": [variance_inflation_factor(X_with_constant.values, i) for i in range(X_with_constant.shape[1])]
})

# Step 5: Afficher les résultats du VIF
print(vif_data)

            Variable          VIF
0              const  3318.782196
1             ln_GDP   219.155667
2           ln_GDP_2   212.377844
3             ln_POP     5.993360
4       urban_growth     1.085124
5      EPS_GHG_indus     2.888642
6  EPS_GHG_transport     2.548672
7   EPS_GHG_building     2.465804
8       EPS_GHG_elec     1.783842


In [100]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Step 1: Define independent variables
variables_of_interest = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth',  
                                'EPS_GHG_indus',
                                #     'EPS_GHG_transport', 'EPS_GHG_building', 'EPS_GHG_elec', 'building_elec', 'transport_indus', 'indus_building',                        
                                'elec_indus', 'transport_elec', 'transport_building',
                                # 'ETS_B', 'ETS_E',
                                # 'ETS_I',  'ETS_T',  'eps_ets_t'
                                # 'eps_ets_b', 'eps_ets_e', 'eps_ets_i',
                                ]


# Step 3: Apply transformations for fixed effects demeaning
vif_data = (
    modGHG[variables_of_interest].copy()  # Create a copy for safe transformations
    .assign( 
        **{col: lambda df, col=col: df[col] - df.groupby('iso3')[col].transform('mean') for col in variables_of_interest}

    )
    .assign( 
        **{col: lambda df, col=col: df[col] - df.groupby('Year')[col].transform('mean') for col in variables_of_interest}

    )
    # .pipe(lambda df: df.loc[:, df.std() > 1e-8])  # Drop near-zero variance columns
    .pipe(lambda df: sm.add_constant(df))  # Add constant for VIF computation
    .pipe(  # Compute VIF
        lambda df: pd.DataFrame({
            "Variable": df.columns,
            "VIF": [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
        })
    )
)


# Step 4: Display VIF results
print(vif_data)


             Variable        VIF
0               const   1.000000
1              ln_GDP  25.203892
2            ln_GDP_2  25.356851
3              ln_POP   1.215313
4        urban_growth   1.130518
5       EPS_GHG_indus   2.378124
6          elec_indus   4.328050
7      transport_elec   2.861976
8  transport_building   1.854266


In [300]:
### Computation of total average effects
# Extraire les coefficients
coefficients = fe_res_with_interaction.params

# Calculer l'effet total
# Suppose coefficients are extracted as:
# beta_elec = coefficients['EPS_GHG_indus']
beta_transport_elec = coefficients['transport_elec']
# beta_building = coefficients['EPS_GHG_building']
beta_indus = coefficients['EPS_GHG_indus']
# beta_transport = coefficients['EPS_GHG_transport']
beta_elec_indus = coefficients['elec_indus']
beta_transport_building = coefficients['transport_building']

# Calculer les contributions individuelles

modGHG = (modGHG
        #   .assign(effect_elec=lambda df: beta_elec * df['EPS_GHG_elec'])
          .assign(effect_transport_elec=lambda df: beta_transport_elec * (df['EPS_GHG_elec'] * df['EPS_GHG_transport']))
        #   .assign(effect_building=lambda df: beta_building * df['EPS_GHG_building'])
          .assign(effect_indus=lambda df: beta_indus * df['EPS_GHG_indus'])
        #   .assign(effect_transport=lambda df: beta_transport * df['EPS_GHG_transport'])
          .assign(effect_elec_indus=lambda df: beta_elec_indus * (df['EPS_GHG_elec'] * df['EPS_GHG_indus']))
          .assign(effect_transport_building=lambda df: beta_transport_building * (df['EPS_GHG_building'] * df['EPS_GHG_transport']))
         )


# Comparer les contributions
# average_effect_elec = modGHG['effect_elec'].mean()
# average_effect_building = modGHG['effect_building'].mean()
average_effect_indus = modGHG['effect_indus'].mean()
# average_effect_transport = modGHG['effect_transport'].mean()
average_effect_transport_elec = modGHG['effect_transport_elec'].mean()
average_effect_elec_indus = modGHG['effect_elec_indus'].mean()
average_effect_transport_building = modGHG['effect_transport_building'].mean()

# print("Average Effect of EPS_GHG_elec:", average_effect_elec)
# print("Average Effect of EPS_GHG_building:", average_effect_building)
# print("Average Effect of EPS_GHG_transport:", average_effect_transport)
print("Average Effect of EPS_GHG_indus:", average_effect_indus)
print("Average Effect of transport_elec:", average_effect_transport_elec)
print("Average Effect of elec_indus:", average_effect_elec_indus)
print("Average Effect of transport_building:", average_effect_transport_building)

Average Effect of EPS_GHG_indus: -0.01829060313750438
Average Effect of transport_elec: -0.046699173959970725
Average Effect of elec_indus: 0.013713477670798108
Average Effect of transport_building: -0.03321856705869196


### GDP

#### Total, Sector, Without elec

In [374]:
exog_var = [
    'ln_GDP',
    'ln_GDP_2',
    'ln_POP', 
    # 'ln_GDP_per_cap',
    # 'ln_GDP_per_cap_2',
    # 'POP_growth',   
    'urban_growth',
    'EPS_GDP',
    # 'ln_EPS_GDP',
    # 'EPS_GDP_building',
    # 'EPS_GDP_elec',
    # 'EPS_GDP_indus',
    # 'EPS_GDP_transport',
    # 'ln_EPS_GDP_building',
    # 'ln_EPS_GDP_elec',
    # 'ln_EPS_GDP_indus',
    # 'ln_EPS_GDP_transport'
    # 'EPS_GDP_building_lag',
    # 'EPS_GDP_elec_lag',
    # 'EPS_GDP_indus_lag',
    # 'EPS_GDP_transport_lag',
    # 'EPS_GDP_building_mbi',
    # 'EPS_GDP_building_nmbi', 
    # 'EPS_GDP_elec_mbi',
    # 'EPS_GDP_elec_nmbi',
    # 'EPS_GDP_indus_mbi',
    # 'EPS_GDP_indus_nmbi',
    # 'EPS_GDP_transport_mbi',
    # 'EPS_GDP_transport_nmbi', 
    # 'EPS_GDP_mbi_mbi',
    # 'EPS_GDP_nmbi_nmbi',
    # 'ETS_E',
    # 'ETS_T',
    # 'ETS_B',
    # 'ETS_I',
    # 'ETS'    
]


exog = sm.add_constant(modGDP[exog_var])
endog= modGDP['ln_building_GHG']
model_fe = PanelOLS(endog, exog, entity_effects = True, time_effects = True) 
fe_res = model_fe.fit(cov_type='clustered') 
print(fe_res)


Inputs contain missing values. Dropping rows with missing observations.


                          PanelOLS Estimation Summary                           
Dep. Variable:        ln_building_GHG   R-squared:                        0.1777
Estimator:                   PanelOLS   R-squared (Between):              0.8724
No. Observations:                1124   R-squared (Within):               0.0356
Date:                Mon, Feb 17 2025   R-squared (Overall):              0.8636
Time:                        13:36:24   Log-likelihood                    359.40
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      45.134
Entities:                          45   P-value                           0.0000
Avg Obs:                       24.978   Distribution:                  F(5,1044)
Min Obs:                       1.0000                                           
Max Obs:                       31.000   F-statistic (robust):             19.542
                            

In [269]:
modGDP['GHG_per_GDP'] = modGDP['GHG'] / modGDP['GDP']*1000
modGDP['ln_GHG_per_GDP'] = np.log(modGDP['GHG_per_GDP'])
modGDP['transport*elec'] = modGDP['EPS_GDP_transport'] * modGDP['EPS_GDP_elec']

# Liste des variables exogènes
exog_var = [
    'GDP_growth',
    'GDP_growth_2',
    'ln_POP', 
    'urban_growth',
    # 'EPS_GDP'
    'EPS_GDP_building', 'EPS_GDP_elec', 'EPS_GDP_indus', 'EPS_GDP_transport'
]

# Ajouter une constante aux variables exogènes
exog = sm.add_constant(modGDP[exog_var])

# Mettre à jour la variable endogène
endog = modGDP['ln_GHG_per_GDP']

# Estimer le modèle avec effets fixes
model_fe = PanelOLS(endog, exog, entity_effects=True, time_effects=True)
fe_res = model_fe.fit(cov_type='clustered')

print(fe_res)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

Inputs contain missing values. Dropping rows with missing observations.


                          PanelOLS Estimation Summary                           
Dep. Variable:         ln_GHG_per_GDP   R-squared:                        0.2281
Estimator:                   PanelOLS   R-squared (Between):             -0.2016
No. Observations:                1098   R-squared (Within):               0.1957
Date:                Mon, Feb 17 2025   R-squared (Overall):             -0.3841
Time:                        10:39:06   Log-likelihood                    1088.6
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      37.523
Entities:                          45   P-value                           0.0000
Avg Obs:                       24.400   Distribution:                  F(8,1016)
Min Obs:                       1.0000                                           
Max Obs:                       30.000   F-statistic (robust):             28.956
                            

#### With interactions

In [16]:
modGDP = (modGDP
          .assign(building_elec=lambda df: df['EPS_GDP_building'] * df['EPS_GDP_elec'])
          .assign(transport_indus=lambda df: df['EPS_GDP_transport'] * df['EPS_GDP_indus'])
          .assign(elec_indus=lambda df: df['EPS_GDP_elec'] * df['EPS_GDP_indus'])
          .assign(transport_building=lambda df: df['EPS_GDP_transport'] * df['EPS_GDP_building'])
          .assign(transport_elec=lambda df: df['EPS_GDP_transport'] * df['EPS_GDP_elec'])
          .assign(indus_building=lambda df: df['EPS_GDP_indus'] * df['EPS_GDP_building'])
          .assign(eps_ets=lambda df: df['EPS_GDP'] * df['ETS'])
          .assign(eps_ets_b=lambda df: df['EPS_GDP_building'] * df['ETS_B'])
          .assign(eps_ets_i=lambda df: df['EPS_GDP_indus'] * df['ETS_I'])
          .assign(eps_ets_e=lambda df: df['EPS_GDP_elec'] * df['ETS_E'])
          .assign(eps_ets_t=lambda df: df['EPS_GDP_transport'] * df['ETS_T'])
         )

# Liste des variables exogènes incluant l'interaction
exog_vars_with_interaction = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth',  
                                'EPS_GDP_building', 'EPS_GDP_transport', 'EPS_GDP_indus',   'EPS_GDP_elec', 
                                #     'eps_ets_b',                      
                                'elec_indus', 'transport_elec', 'transport_building',
                                # 'ETS_B', 'ETS_T',  'eps_ets_t'  'eps_ets_e', 'building_elec', 'transport_indus', 'indus_building', 
                                # 'ETS_I', 'ETS_E',  
                                #  'eps_ets',
                                ]
# exog_vars_with_interaction = ['ln_GDP', 'ln_POP', 'urban_growth',
                            #    'EPS_GDP', 
                            #    'ETS', 
                            #    'eps_ets'
                            #    ]


# Construire les variables exogènes et endogènes
exog_with_interaction = sm.add_constant(modGDP[exog_vars_with_interaction])
endog = modGDP['ln_GHG']

# Modèle avec effets fixes
model_fe_with_interaction = PanelOLS(endog, exog_with_interaction, entity_effects=True, time_effects=True) 
fe_res_with_interaction = model_fe_with_interaction.fit(cov_type='clustered') 
print(fe_res_with_interaction)


Inputs contain missing values. Dropping rows with missing observations.


                          PanelOLS Estimation Summary                           
Dep. Variable:                 ln_GHG   R-squared:                        0.5979
Estimator:                   PanelOLS   R-squared (Between):              0.9373
No. Observations:                1147   R-squared (Within):               0.4904
Date:                Fri, Feb 21 2025   R-squared (Overall):              0.9295
Time:                        16:04:06   Log-likelihood                    1338.4
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      143.28
Entities:                          46   P-value                           0.0000
Avg Obs:                       24.935   Distribution:                 F(11,1060)
Min Obs:                       1.0000                                           
Max Obs:                       31.000   F-statistic (robust):             113.24
                            

##### Tests interactions

In [21]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Step 1: Sélectionner les variables d'intérêt, y compris 'Year'
variables_of_interest = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth',  
                                'EPS_GDP_building', 'EPS_GDP_transport', 'EPS_GDP_indus',   'EPS_GDP_elec', 
                                #     'eps_ets_b',                      
                                # 'building_elec', 'transport_indus', 'indus_building', 'elec_indus', 'transport_elec', 'transport_building',
                                # 'ETS_B', 'ETS_T',  'eps_ets_t'  'eps_ets_e',
                                # 'ETS_I', 'ETS_E',  
                                 'eps_ets',
                                ]
                            

# Step 2: Nettoyer les données
# Remplacer les valeurs infinies par NaN, puis supprimer les lignes avec des NaN dans les variables d'intérêt
modGDP_cleaned = (
    modGDP[variables_of_interest]  # On garde uniquement les colonnes d'intérêt
    .replace([np.inf, -np.inf], np.nan)  # Remplacer les infinis par NaN
    .dropna()  # Supprimer les lignes avec des NaN
)

# Step 3: Ajouter une constante pour le calcul du VIF
X_with_constant = sm.add_constant(modGDP_cleaned)

# Step 4: Calculer le VIF pour chaque variable, y compris 'Year'
vif_data = pd.DataFrame({
    "Variable": X_with_constant.columns,
    "VIF": [variance_inflation_factor(X_with_constant.values, i) for i in range(X_with_constant.shape[1])]
})

# Step 5: Afficher les résultats du VIF
print(vif_data)

            Variable          VIF
0              const  3349.012387
1             ln_GDP   218.035416
2           ln_GDP_2   213.013528
3             ln_POP     5.304151
4       urban_growth     1.203254
5   EPS_GDP_building     3.018995
6  EPS_GDP_transport     3.279993
7      EPS_GDP_indus     2.768273
8       EPS_GDP_elec     1.433493
9            eps_ets     4.711471


In [17]:
#### avec demean

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Step 1: Define independent variables
variables_of_interest = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth',  
                            #  'eps_ets_b', 'building_elec', 'transport_indus', 'indus_building',                      
                            'EPS_GDP_indus','EPS_GDP_building', 'EPS_GDP_transport',  'EPS_GDP_elec', 'elec_indus', 'transport_elec', 'transport_building', 
                            # 'ETS_B', 'ETS_T',  'eps_ets_t'  'eps_ets_e',
                            # 'ETS_I', 'ETS_E',  
                            # 'eps_ets',
                            ]


# Step 3: Apply transformations for fixed effects demeaning
vif_data = (
    modGDP[variables_of_interest].copy()  # Create a copy for safe transformations
    .assign( 
        **{col: lambda df, col=col: df[col] - df.groupby('iso3')[col].transform('mean') for col in variables_of_interest}

    )
    .assign( 
        **{col: lambda df, col=col: df[col] - df.groupby('Year')[col].transform('mean') for col in variables_of_interest}

    )
    # .pipe(lambda df: df.loc[:, df.std() > 1e-8])  # Drop near-zero variance columns
    .pipe(lambda df: sm.add_constant(df))  # Add constant for VIF computation
    .pipe(  # Compute VIF
        lambda df: pd.DataFrame({
            "Variable": df.columns,
            "VIF": [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
        })
    )
)


# Step 4: Display VIF results
print(vif_data)


MissingDataError: exog contains inf or nans

In [296]:
### Computation of total average effects
# Extraire les coefficients
coefficients = fe_res_with_interaction.params

# Calculer l'effet total
# Suppose coefficients are extracted as:
# beta_elec = coefficients['EPS_GDP_indus']
beta_transport_elec = coefficients['transport_elec']
# beta_building = coefficients['EPS_GDP_building']
beta_indus = coefficients['EPS_GDP_indus']
# beta_transport = coefficients['EPS_GDP_transport']
beta_elec_indus = coefficients['elec_indus']
beta_transport_building = coefficients['transport_building']

# Calculer les contributions individuelles

modGDP = (modGDP
        #   .assign(effect_elec=lambda df: beta_elec * df['EPS_GDP_elec'])
          .assign(effect_transport_elec=lambda df: beta_transport_elec * (df['EPS_GDP_elec'] * df['EPS_GDP_transport']))
        #   .assign(effect_building=lambda df: beta_building * df['EPS_GDP_building'])
          .assign(effect_indus=lambda df: beta_indus * df['EPS_GDP_indus'])
        #   .assign(effect_transport=lambda df: beta_transport * df['EPS_GDP_transport'])
          .assign(effect_elec_indus=lambda df: beta_elec_indus * (df['EPS_GDP_elec'] * df['EPS_GDP_indus']))
          .assign(effect_transport_building=lambda df: beta_transport_building * (df['EPS_GDP_building'] * df['EPS_GDP_transport']))
         )


# Comparer les contributions
# average_effect_elec = modGDP['effect_elec'].mean()
# average_effect_building = modGDP['effect_building'].mean()
average_effect_indus = modGDP['effect_indus'].mean()
# average_effect_transport = modGDP['effect_transport'].mean()
average_effect_transport_elec = modGDP['effect_transport_elec'].mean()
average_effect_elec_indus = modGDP['effect_elec_indus'].mean()
average_effect_transport_building = modGDP['effect_transport_building'].mean()

# print("Average Effect of EPS_GDP_elec:", average_effect_elec)
# print("Average Effect of EPS_GDP_building:", average_effect_building)
# print("Average Effect of EPS_GDP_transport:", average_effect_transport)
print("Average Effect of EPS_GDP_indus:", average_effect_indus)
print("Average Effect of transport_elec:", average_effect_transport_elec)
print("Average Effect of elec_indus:", average_effect_elec_indus)
print("Average Effect of transport_building:", average_effect_transport_building)

Average Effect of EPS_GDP_indus: -0.06701940537963857
Average Effect of transport_elec: -0.03222349945297896
Average Effect of elec_indus: 0.03325454791974547
Average Effect of transport_building: -0.05273241843842836


#### GHG/GDP

In [ ]:
modGDP['GHG_per_GDP'] = modGDP['GHG'] / modGDP['GDP']
modGDP['ln_GHG_per_GDP'] = np.log(modGDP['GHG_per_GDP'])
modGDP['transport*elec'] = modGDP['EPS_GDP_transport'] * modGDP['EPS_GDP_elec']

# Liste des variables exogènes
exog_var = [
    'ln_POP', 
    'urban_growth',
    'EPS_GDP'
    # 'EPS_GDP_building', 'EPS_GDP_elec', 'EPS_GDP_indus', 'EPS_GDP_transport'
]

# Ajouter une constante aux variables exogènes
exog = sm.add_constant(modGDP[exog_var])

# Mettre à jour la variable endogène
endog = modGDP['GHG_per_GDP']

# Estimer le modèle avec effets fixes
model_fe = PanelOLS(endog, exog, entity_effects=True, time_effects=True)
fe_res = model_fe.fit(cov_type='clustered')

print(fe_res)

## Diagnosis and Model choice

### Pooled versus Panel

In [11]:
exog_var = [
    'ln_GDP',
    'ln_GDP_2',
    'ln_POP', 
    # 'ln_GDP_per_cap',
    # 'POP_growth',   
    'urban_growth',
    # 'EPS_OCDE',
    'EPS_OCDE_building',
    'EPS_OCDE_elec',
    'EPS_OCDE_indus',
    'EPS_OCDE_transport'
    # 'EPS_OCDE_building_lag',
    # 'EPS_OCDE_elec_lag',
    # 'EPS_OCDE_indus_lag',
    # 'EPS_OCDE_transport_lag',
    # 'EPS_OCDE_building_mbi',
    # 'EPS_OCDE_building_nmbi', 
    # 'EPS_OCDE_elec_mbi',
    # 'EPS_OCDE_elec_nmbi',
    # 'EPS_OCDE_indus_mbi',
    # 'EPS_OCDE_indus_nmbi',
    # 'EPS_OCDE_transport_mbi',
    # 'EPS_OCDE_transport_nmbi', 
    # 'EPS_OCDE_mbi_mbi',
    # 'EPS_OCDE_nmbi_nmbi',
       
]


exog = sm.add_constant(modOCDE[exog_var])
endog= modOCDE['ln_GHG']
mod = PooledOLS(endog, exog)
pooled_res = mod.fit(cov_type='clustered')
print(pooled_res)
# Store values for checking homoskedasticity graphically
fittedvals_pooled_OLS = pooled_res.predict().fitted_values
residuals_pooled_OLS = pooled_res.resids

                          PooledOLS Estimation Summary                          
Dep. Variable:                 ln_GHG   R-squared:                        0.9408
Estimator:                  PooledOLS   R-squared (Between):              0.9485
No. Observations:                1152   R-squared (Within):               0.3649
Date:                Sun, Feb 16 2025   R-squared (Overall):              0.9408
Time:                        11:33:13   Log-likelihood                   -486.91
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2271.4
Entities:                          46   P-value                           0.0000
Avg Obs:                       25.043   Distribution:                  F(8,1143)
Min Obs:                       1.0000                                           
Max Obs:                       31.000   F-statistic (robust):             3089.4
                            

In [ ]:
# 3A.1 Residuals-Plot for growing Variance Detection
fig, ax = plt.subplots()
ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS, color = 'blue')
ax.axhline(0, color = 'r', ls = '--')
ax.set_xlabel('Predicted Values', fontsize = 15)
ax.set_ylabel('Residuals', fontsize = 15)
ax.set_title('Homoskedasticity Test', fontsize = 30)

plt.show()

In [12]:
# 3A.2 White-Test
pooled_OLS_dataset = pd.concat([modOCDE, residuals_pooled_OLS], axis=1)
pooled_OLS_dataset = pooled_OLS_dataset.drop(['Year', 'iso3'], axis = 1).fillna(0)
exog_vars = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'EPS_OCDE_building', 'EPS_OCDE_elec', 'EPS_OCDE_indus', 'EPS_OCDE_transport']
exog = sm.add_constant(modOCDE[exog_vars]).fillna(0)


white_test_results = het_white(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, white_test_results)))

# 3A.3 Breusch-Pagan-Test
breusch_pagan_test_results = het_breuschpagan(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, breusch_pagan_test_results)))

### p-value small so 3.A violated -> presence of heterogeneity

{'LM-Stat': np.float64(406.7713424521279), 'LM p-val': np.float64(2.0826664466705084e-65), 'F-Stat': np.float64(17.93226126035723), 'F p-val': np.float64(7.561963300286663e-83)}
{'LM-Stat': np.float64(28.25416157309928), 'LM p-val': np.float64(0.00019775048890963032), 'F-Stat': np.float64(4.109058387497663), 'F p-val': np.float64(0.00018070720291612304)}


In [13]:
# 3.B Non-Autocorrelation
# Durbin-Watson-Test
from statsmodels.stats.stattools import durbin_watson

durbin_watson_test_results = durbin_watson(pooled_OLS_dataset['residual']) 
print(durbin_watson_test_results)

## the mean (2) indicate no autocorrelation, 
## For GDP, 2.7 small negative correlation 
### for OECD reference , 2.27, small négative correlation 
### 

2.20490797393706


### Fixed vs random

##### Generalized Hausman

In [14]:
exog_var = [
    'ln_GDP',
    'ln_GDP_2',
    'ln_POP',
    'urban_growth',
    'EPS_OCDE_building',
    'EPS_OCDE_elec',
    'EPS_OCDE_indus',
    'EPS_OCDE_transport'
]

exog = sm.add_constant(modOCDE[exog_var])
endog = modBOD['ln_GHG']

# Ajuster le modèle à effets aléatoires (RE)
model_re = RandomEffects(endog, exog)
re_res = model_re.fit(cov_type='clustered')

# Ajuster le modèle à effets fixes (FE)
model_fe = PanelOLS(endog, exog, entity_effects = True, time_effects = True)
fe_res = model_fe.fit(cov_type='clustered')

# Afficher les paramètres et les matrices de covariance des deux modèles
print("Paramètres du modèle FE:")
print(fe_res.params)

print("\nParamètres du modèle RE:")
print(re_res.params)

print("\nMatrice de covariance du modèle FE:")
print(fe_res.cov)

print("\nMatrice de covariance du modèle RE:")
print(re_res.cov)

# Vérifier les différences de paramètres
diff_params =  fe_res.params - re_res.params
print("\nDifférence des paramètres:")
print(diff_params)

# Hausman test for endogeneity
def hausman(fe, re):
    B = fe.params
    b = re.params
    v_B = fe.cov
    v_b = re.cov

    # Ensure the indices are aligned for comparison
    common_idx = B.index.intersection(b.index)
    B = B[common_idx]
    b = b[common_idx]
    v_B = v_B.loc[common_idx, common_idx]
    v_b = v_b.loc[common_idx, common_idx]

    # Check if the difference matrix is positive definite
    diff_matrix = v_B - v_b
    eigvals = np.linalg.eigvals(diff_matrix)
    print("\nEigenvalues of the difference matrix:", eigvals)
    diff_matrix_inv = la.pinv(diff_matrix)
    df = len(common_idx)
    chi2 = np.dot((b - B).T, diff_matrix_inv.dot(b - B))
    pval = stats.chi2.sf(chi2, df)

    return chi2, df, pval

hausman_results = hausman(fe_res, re_res)
print('chi-Squared: ' + str(hausman_results[0]))
print('degrees of freedom: ' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

Paramètres du modèle FE:
const                 2.764769
ln_GDP               -0.487261
ln_GDP_2              0.043702
ln_POP                0.503511
urban_growth         -0.047938
EPS_OCDE_building     0.012019
EPS_OCDE_elec        -0.074827
EPS_OCDE_indus       -0.123944
EPS_OCDE_transport   -0.109268
Name: parameter, dtype: float64

Paramètres du modèle RE:
const                 3.140018
ln_GDP               -0.481011
ln_GDP_2              0.038870
ln_POP                0.525952
urban_growth         -0.047363
EPS_OCDE_building    -0.031008
EPS_OCDE_elec        -0.097304
EPS_OCDE_indus       -0.095675
EPS_OCDE_transport   -0.134424
Name: parameter, dtype: float64

Matrice de covariance du modèle FE:
                       const    ln_GDP  ln_GDP_2        ln_POP  urban_growth  \
const               2.330412 -0.149458  0.006152 -8.771023e-02 -6.054057e-04   
ln_GDP             -0.149458  0.016257 -0.000663  3.107643e-03  9.917036e-05   
ln_GDP_2            0.006152 -0.000663  0.000029 -

#### multicolinearity and correlation

In [51]:
###### Because strange restult on time fixed effects, we test the colinearity # Step-by-step with chaining
# Step 1: Define the variables of interest, including 'Year' as a continuous variable
variables_of_interest = [
    'ln_GDP',
    'ln_GDP_2',
    'ln_POP',
    'urban_growth',
    'ln_GHG',
    'EPS_GDP',
    'Year'
]

# Step 2: Select the columns for correlation analysis
correlation_matrix = modGDP[variables_of_interest].corr()

# Step 3: Display the correlation matrix
print(correlation_matrix)

                ln_GDP  ln_GDP_2    ln_POP  urban_growth    ln_GHG   EPS_GDP  \
ln_GDP        1.000000  0.997535  0.863507      0.143725  0.908481  0.255682   
ln_GDP_2      0.997535  1.000000  0.860721      0.134881  0.904730  0.253077   
ln_POP        0.863507  0.860721  1.000000      0.073409  0.949678  0.081976   
urban_growth  0.143725  0.134881  0.073409      1.000000  0.051737 -0.049299   
ln_GHG        0.908481  0.904730  0.949678      0.051737  1.000000  0.085863   
EPS_GDP       0.255682  0.253077  0.081976     -0.049299  0.085863  1.000000   
Year          0.065998  0.066866  0.023868      0.003590 -0.024166  0.834552   

                  Year  
ln_GDP        0.065998  
ln_GDP_2      0.066866  
ln_POP        0.023868  
urban_growth  0.003590  
ln_GHG       -0.024166  
EPS_GDP       0.834552  
Year          1.000000  


In [ ]:

from statsmodels.stats.outliers_influence import variance_inflation_factor
# Step 1: Sélectionner les variables d'intérêt, y compris 'Year'
variables_of_interest = ['ln_GDP', 'ln_GDP_2', 'ln_POP', 'urban_growth', 'EPS_OCDE_building', 'EPS_OCDE_elec', 'EPS_OCDE_indus', 'EPS_OCDE_transport', 'Year']

# Step 2: Nettoyer les données
# Remplacer les valeurs infinies par NaN, puis supprimer les lignes avec des NaN dans les variables d'intérêt
mod_cleaned = (
    modOCDE[variables_of_interest]  # On garde uniquement les colonnes d'intérêt
    .replace([np.inf, -np.inf], np.nan)  # Remplacer les infinis par NaN
    .dropna()  # Supprimer les lignes avec des NaN
)

# Step 3: Ajouter une constante pour le calcul du VIF
X_with_constant = sm.add_constant(mod_cleaned)

# Step 4: Calculer le VIF pour chaque variable, y compris 'Year'
vif_data = pd.DataFrame({
    "Variable": X_with_constant.columns,
    "VIF": [variance_inflation_factor(X_with_constant.values, i) for i in range(X_with_constant.shape[1])]
})

# Step 5: Afficher les résultats du VIF
print(vif_data)


##### multicolinearité modérée, sous le seuil de 10 pour toutes les variables. on est contents

             Variable            VIF
0               const  264287.515710
1              ln_GDP     222.595146
2            ln_GDP_2     214.999334
3              ln_POP       6.087407
4        urban_growth       1.190426
5       EPS_OCDE_elec       5.672147
6      EPS_OCDE_indus       5.093771
7  EPS_OCDE_transport       3.854926
8                Year       4.209386


#### BIC and AIC

In [ ]:
exog_var_1 = [
    'ln_GDP',
    'ln_POP',
    'urban_growth',
    'EPS_OCDE_building',
    'EPS_OCDE_elec',
    'EPS_OCDE_indus',
    'EPS_OCDE_transport'
]

exog_var_2 = [
    'ln_GDP',
    'ln_GDP_2',
    'ln_POP',
    'urban_growth',
    'EPS_OCDE_building',
    'EPS_OCDE_elec',
    'EPS_OCDE_indus',
    'EPS_OCDE_transport'
]

exog1 = sm.add_constant(modOCDE[exog_var_1])
exog2 = sm.add_constant(modOCDE[exog_var_2])
endog = modOCDE['ln_GHG']

model1 = PanelOLS(endog, exog1, entity_effects=True, time_effects=True)
fe_1 = model1.fit(cov_type='clustered')

model2 = PanelOLS(endog, exog2, entity_effects=True, time_effects=True)
fe_2 = model2.fit(cov_type='clustered')


def calc_aic_manual(result, n):
    # Extraire les résidus
    residuals = result.resids
    rss = np.sum(residuals**2)  # Calcul du RSS (Residual Sum of Squares)
    
    # Calcul de la log-vraisemblance (approximative)
    log_likelihood = -n / 2 * (np.log(2 * np.pi) + 1 + np.log(rss / n))
    
    # Nombre de paramètres estimés
    num_params = len(result.params)
    
    # Calcul de l'AIC et du bic 
    aic = -2 * log_likelihood + 2 * num_params
    return aic

# Fonction pour calculer l'AIC manuellement
def calc_bic_manual(result, n):
    # Extraire les résidus
    residuals = result.resids
    rss = np.sum(residuals**2)  # Calcul du RSS (Residual Sum of Squares)
    
    # Calcul de la log-vraisemblance (approximative)
    log_likelihood = -n / 2 * (np.log(2 * np.pi) + 1 + np.log(rss / n))
    
    # Nombre de paramètres estimés
    num_params = len(result.params)
    
    # Calcul de l'AIC et du bic 
    bic = -2 * log_likelihood + np.log(n)* num_params
    return bic

# Nombre d'observations (n)
n = endog.shape[0]

# Calcul de l'AIC et du bic pour chaque modèle

aic_1 = calc_aic_manual(fe_1, n)
aic_2 = calc_aic_manual(fe_2, n)


bic_1 = calc_bic_manual(fe_1, n)
bic_2 = calc_bic_manual(fe_2, n)

# Afficher les résultats
print(f"AIC normal: {aic_1}")
print(f"AIC avec GDP2: {aic_2}")


print(f"bic normal: {bic_1}")
print(f"bic avec GDP2: {bic_2}")

In [15]:
exog_var = [
    'ln_GDP',
    'ln_GDP_2',
    'ln_POP',
    'urban_growth',
    'EPS_OCDE_building',
    'EPS_OCDE_elec',
    'EPS_OCDE_indus',
    'EPS_OCDE_transport'
]

exog = sm.add_constant(modOCDE[exog_var])
endog = modOCDE['ln_GHG']

# Ajuster le modèle à effets aléatoires (RE)
model_re = RandomEffects(endog, exog)
re_res = model_re.fit(cov_type='clustered')

model_entity_effects = PanelOLS(endog, exog, entity_effects=True)
fe_entity_res = model_entity_effects.fit(cov_type='clustered')

# Modèle à effets fixes temporels uniquement
model_time_effects = PanelOLS(endog, exog, time_effects=True)
fe_time_res = model_time_effects.fit(cov_type='clustered')

# Modèle avec à la fois des effets d'entités et des effets temporels
model_entity_time_effects = PanelOLS(endog, exog, entity_effects=True, time_effects=True)
fe_entity_time_res = model_entity_time_effects.fit(cov_type='clustered')

# Comparaison entre les différents modèles
comparison = compare({'RE': re_res,
                      'Entity': fe_entity_res,
                      'Time': fe_time_res,
                      'Entity/Time': fe_entity_time_res})

print(comparison)
#### Entity effects not good (lower significativity on every levels),
#### time more ambigous because strong f-stat but R2 within negative, as well as log-likelihood

# Fonction pour calculer l'AIC manuellement
def calc_aic_manual(result, n):
    # Extraire les résidus
    residuals = result.resids
    rss = np.sum(residuals**2)  # Calcul du RSS (Residual Sum of Squares)
    
    # Calcul de la log-vraisemblance (approximative)
    log_likelihood = -n / 2 * (np.log(2 * np.pi) + 1 + np.log(rss / n))
    
    # Nombre de paramètres estimés
    num_params = len(result.params)
    
    # Calcul de l'AIC et du bic 
    aic = -2 * log_likelihood + 2 * num_params
    return aic

# Fonction pour calculer l'AIC manuellement
def calc_bic_manual(result, n):
    # Extraire les résidus
    residuals = result.resids
    rss = np.sum(residuals**2)  # Calcul du RSS (Residual Sum of Squares)
    
    # Calcul de la log-vraisemblance (approximative)
    log_likelihood = -n / 2 * (np.log(2 * np.pi) + 1 + np.log(rss / n))
    
    # Nombre de paramètres estimés
    num_params = len(result.params)
    
    # Calcul de l'AIC et du bic 
    bic = -2 * log_likelihood + np.log(n)* num_params
    return bic

# Nombre d'observations (n)
n = endog.shape[0]

# Calcul de l'AIC et du bic pour chaque modèle
aic_re = calc_aic_manual(re_res, n)
aic_entity = calc_aic_manual(fe_entity_res, n)
aic_time = calc_aic_manual(fe_time_res, n)
aic_entity_time = calc_aic_manual(fe_entity_time_res, n)

bic_re = calc_bic_manual(re_res, n)
bic_entity = calc_bic_manual(fe_entity_res, n)
bic_time = calc_bic_manual(fe_time_res, n)
bic_entity_time = calc_bic_manual(fe_entity_time_res, n)

# Afficher les résultats
print(f"AIC pour le modèle avec effets aleatoire: {aic_re}")
print(f"AIC pour le modèle avec effets d'entités: {aic_entity}")
print(f"AIC pour le modèle avec effets temporels: {aic_time}")
print(f"AIC pour le modèle avec effets d'entités et temporels: {aic_entity_time}")

print(f"bic pour le modèle avec effets aleatoire: {bic_re}")
print(f"bic pour le modèle avec effets d'entités: {bic_entity}")
print(f"bic pour le modèle avec effets temporels: {bic_time}")
print(f"bic pour le modèle avec effets d'entités et temporels: {bic_entity_time}")



                                  Model Comparison                                 
                                       RE        Entity          Time   Entity/Time
-----------------------------------------------------------------------------------
Dep. Variable                      ln_GHG        ln_GHG        ln_GHG        ln_GHG
Estimator                   RandomEffects      PanelOLS      PanelOLS      PanelOLS
No. Observations                     1152          1152          1152          1152
Cov. Est.                       Clustered     Clustered     Clustered     Clustered
R-squared                          0.7758        0.5385        0.9409        0.5462
R-Squared (Within)                 0.5380        0.5385        0.1821        0.4298
R-Squared (Between)                0.9382        0.9331        0.9489        0.9312
R-Squared (Overall)                0.9300        0.9256        0.9402        0.9186
F-statistic                        494.34        160.14        2216.4       

### Choice of covariance estimator

In [16]:
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey

# Fit the FE model
exog_var = [
    'ln_GDP',
    'ln_GDP_2',
    'ln_POP',
    'urban_growth',
    'EPS_OCDE_building',
    'EPS_OCDE_elec',
    'EPS_OCDE_indus',
    'EPS_OCDE_transport'
]

exog = sm.add_constant(modOCDE[exog_var])
endog = modOCDE['ln_GHG']

model_fe = PanelOLS(endog, exog, entity_effects=True, time_effects= True)
fe_res = model_fe.fit(cov_type='unadjusted')

### 1. Heteroskedasticity Test (Breusch-Pagan)
# Breusch-Pagan requires the residuals and the independent variables (exog)
_, bp_pvalue, _, _ = het_breuschpagan(fe_res.resids, exog)
print(f"Breusch-Pagan test for heteroskedasticity - p-value: {bp_pvalue}")
if bp_pvalue < 0.05:
    print("There is evidence of heteroskedasticity (use robust or clustered standard errors).")
else:
    print("No evidence of heteroskedasticity.")


Breusch-Pagan test for heteroskedasticity - p-value: 2.981490897531128e-12
There is evidence of heteroskedasticity (use robust or clustered standard errors).


In [17]:
### 2. Autocorrelation Test (Breusch-Godfrey)
# Breusch-Godfrey test for serial correlation (autocorrelation)
# Requires the residuals and number of lags (you can try different lags)
# Step 1: Extract residuals from the fitted FE model
from statsmodels.regression.linear_model import OLS 
residuals = fe_res.resids

# Step 2: Perform OLS regression of residuals on lags of residuals (like time-series)
lags = 2  # You can adjust the number of lags
bg_test = acorr_breusch_godfrey(OLS(residuals, exog).fit(), nlags=lags)
print(f"Breusch-Godfrey test for autocorrelation - p-value: {bg_test[1]}")
if bg_test[1] < 0.05:
    print("There is evidence of autocorrelation (use clustered or Driscoll-Kraay standard errors).")
else:
    print("No evidence of autocorrelation.")

Breusch-Godfrey test for autocorrelation - p-value: 1.2685103590033155e-09
There is evidence of autocorrelation (use clustered or Driscoll-Kraay standard errors).


In [18]:
### 3. Cross-Sectional Dependence Test (Pesaran's test)
# Pesaran's test for cross-sectional dependence

# Function to compute Pesaran's Cross-Sectional Dependence (CD) test
def pesaran_cd(residuals):
    """
    Perform Pesaran's test for cross-sectional dependence on residuals.
    
    Parameters:
    residuals (pd.DataFrame or np.ndarray): Residuals from a panel model, where rows represent time periods and columns represent entities.
    
    Returns:
    stat (float): Pesaran's CD test statistic.
    p-value (float): p-value for the test.
    """
    # Check for missing values in residuals
    if residuals.isnull().any().any():
        print("Warning: Missing values found in residuals. These will be dropped.")
        residuals = residuals.dropna()  # Drop rows with missing values

    # Check for constant columns (entities with no variation in residuals)
    constant_columns = residuals.columns[residuals.nunique() == 1]
    if not constant_columns.empty:
        print(f"Warning: Constant residuals detected for entities: {constant_columns}. These will be dropped.")
        residuals = residuals.drop(columns=constant_columns)

    T, N = residuals.shape  # T: Time periods, N: Entities
    residuals = np.asarray(residuals)
    
    # Compute pairwise correlations of residuals across entities
    rho_matrix = np.corrcoef(residuals, rowvar=False)
    
    # Extract the upper triangle (off-diagonal elements) from the correlation matrix
    upper_triangle_indices = np.triu_indices(N, k=1)
    rho_values = rho_matrix[upper_triangle_indices]
    
    # Compute the Pesaran CD test statistic
    pesaran_stat = np.sqrt(2 / (N * (N - 1))) * np.sum(rho_values)
    
    # Compute the p-value using the standard normal distribution
    p_value = 2 * (1 - stats.norm.cdf(np.abs(pesaran_stat)))
    
    return pesaran_stat, p_value


# Example Usage with Panel Data and Residuals

# Fit the FE model (Fixed Effects model)
model_fe = PanelOLS(endog, exog, entity_effects=True)
fe_res = model_fe.fit(cov_type='unadjusted')

# Extract the residuals from the fitted model and reshape to (T x N) format
residuals = fe_res.resids.unstack()  # Convert to DataFrame with time as rows and entities as columns

# Perform Pesaran's test for cross-sectional dependence
pesaran_stat, p_value = pesaran_cd(residuals)

# Output the test results
print(f"Pesaran's CD test statistic: {pesaran_stat}")
print(f"Pesaran's CD test p-value: {p_value}")

# Check if cross-sectional dependence is significant
if p_value < 0.05:
    print("There is evidence of cross-sectional dependence.")
else:
    print("No evidence of cross-sectional dependence.")

Pesaran's CD test statistic: -0.38013833006387243
Pesaran's CD test p-value: 0.7038427343710785
No evidence of cross-sectional dependence.


In [ ]:
# Fit with robust standard errors (to address heteroskedasticity)
fe_res_robust = model_fe.fit(cov_type='robust')

# Fit with clustered standard errors by entity (to address autocorrelation within entities)
fe_res_clustered = model_fe.fit(cov_type='clustered', cluster_entity=True)

# Fit with Driscoll-Kraay standard errors (to address both autocorrelation and cross-sectional dependence)
fe_res_driscoll_kraay = model_fe.fit(cov_type='clustered')

# Compare the different model results
comparison = compare({'FE Unadjusted': fe_res, 
                      'FE Robust': fe_res_robust, 
                      'FE Clustered': fe_res_clustered, 
                      'FE Driscoll-Kraay': fe_res_driscoll_kraay})

# Print the comparison table
print(comparison)

# Other tests if needed